<a href="https://colab.research.google.com/github/kabaaaa-create/master-research/blob/main/firstSNSprototype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio==3.48.0
#!git clone https://github.com/ExponentialML/Text-To-Video-Finetuning.git
!git clone https://github.com/usghdfic/Text-To-Video-Finetuning.git
from google.colab import drive
drive.mount('/content/drive')
%cd Text-To-Video-Finetuning
!pip install -r requirements.txt
!pip install accelerate==0.19.0
!pip install diffusers==0.17.0
!pip install transformers==4.30.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 52.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 12.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 10.1 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 9.5 MB/s eta 0:00:00
  Attempting uninstall: diffusers
    Found existing installation: diffusers 0.23.0.dev0
    Uninstalling diffusers-0.23.0.dev0:
      Successfully uninstalled diffusers-0.23.0.dev0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 50.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 87.5 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.14.1
    Uninstalling tokenizers-0.14.1:
      Successfully uninstalled tokenizers-0.14.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.0
    Uninstalling transformers-4.35.0:
      Successfully uninstalled transformers-4.35.0


In [ ]:
def generate(userid, prompt):
  !python /content/Text-To-Video-Finetuning/inference.py --model "/content/drive/MyDrive/KabaModel2" --prompt "$prompt" --num-frames 24 --num-steps 25 --window-size 12 --width 512 --height 512 --sdp --loop --negative-prompt "typography, people, text, font, phrase, human, character, animals, cat, dog, realistic" -o "/content/videos/" -u $userid
  video = "/content/videos/{}{}.mp4".format(prompt,userid)
  gif = "{}{}.gif".format(prompt,userid)
  !ffmpeg -y -i "$video" "/content/videos/$gif"
  return "/content/videos/{}{}.gif".format(prompt,userid)

def generatefrom(userid, prompt, postid):
  ip = posts[postid]
  basevideo = "{}{}.mp4".format(ip["text"],ip["user"])
  !python /content/Text-To-Video-Finetuning/inference.py --model "/content/drive/MyDrive/KabaModel2" --prompt "$prompt" --num-frames 24 --num-steps 25 --window-size 12 --width 512 --height 512 --sdp --loop --negative-prompt "typography, people, text, font, phrase, human, character, animals, cat, dog, realistic" -o "/content/videos/" -u $userid -i "/content/videos/$basevideo"  -iw 0.1
  video = "/content/videos/{}{}.mp4".format(prompt,userid)
  gif = "{}{}.gif".format(prompt,userid)
  !ffmpeg -y -i "$video" "/content/videos/$gif"
  return "/content/videos/{}{}.gif".format(prompt,userid)

users=[]

def newuser(email,password):
  id = len(users)
  user={"id":id,"email":email,"password":password,"icon":"","post":[],"comment":[],"follower":[],"following":[]}
  users.append(user)
  return user["id"]

import time
posts=[]

def newpost(userid,text):
  # print(userid, text)
  id = len(posts)
  t = time.localtime()
  current_time = time.strftime("%H:%M:%S", t)
  posttest = generate(userid, text)
  post={"id":id,"user":userid,"gif":posttest,"text":text,"time":current_time,"comment":[]}
  #generate function
  posts.append(post)
  users[userid]["post"].append(id)
  return post["id"]

comments=[]

def newcomment(userid,postid,text):
  id = len(comments)
  t = time.localtime()
  current_time = time.strftime("%H:%M:%S", t)
  #posttest = generate(userid, text)
  gif = generatefrom(userid, text, postid)
  comment={"id":id,"user":userid,"postid":postid,"gif":gif,"text":text,"time":current_time}
  #generate function
  comments.append(comment)
  users[userid]["comment"].append(id)
  posts[postid]["comment"].append(id)
  return comment["id"]

def follow(userid,followid):
  notfollowing=True
  if followid in users[userid]["following"]:
    notfollowing=False
  else:
    notfollowing=True
    users[userid]["following"].append(followid)
    users[followid]["follower"].append(userid)
  return notfollowing

from PIL import Image
import json

def home(userid):
  return json.dumps(posts)

def post(userid, postid):
  return json.dumps(posts[postid])

def create(userid, text):
  #atarashii post
  # print("this is the user id")
  # print(userid)
  post = newpost(userid,text)
  return json.dumps(post)

def createcomment(userid,text, postid):
  comment = newcomment(userid,postid,text)
  return json.dumps(comment)

def profile(userid):
  return json.dumps(users[userid])

def following(userid):
  return json.dumps(users[userid]["following"])

def followers(userid):
  return json.dumps(users[userid]["follower"])

def getgif(postid):
  post = posts[postid]
  # image = Image.open("/content/videos/{}{}.gif".format(post["text"],post["user"]))
  return post["gif"]

def getcommentgif(postid):
  post = comments[postid]
  # image = Image.open("/content/videos/{}{}.gif".format(post["text"],post["user"]))
  return post["gif"]

import numpy as np

happy1 = np.array([[0,4832],[0.06999999999999999,5232],[0,5574],[0.11249999999999999,5684],[0,5908],[0.6675,6133],[0,6349],[0,6616]])
happy2 = np.array([[0,4847],[0.5475,5070],[0.155,5190],[0,5190],[0,5593],[0,5853],[0,6080],[0,6358]])
happy3 = np.array([[0,1475],[0.02,1515],[0.21999999999999997,1515],[0,1658],[0.22749999999999998,1735],[0.1075,1839],[0,1948],[0.02,2038],[0.32,2038]])
happy4 = np.array([[0,2504],[0.26,2762],[0.10499999999999998,2872],[0,3054],[0.17500000000000002,3196],[0,3505],[0,3771],[0,3905],[0.6,4362],[0.32249999999999995,4656],[0,4889],[0,5332]])
happy5 = np.array([[0,2994],[0,3394],[0,3530],[0.75,3872],[0,4360],[0.36,4837],[0.1475,4909],[0,5298],[0.2525,5401],[0,5769],[0.19999999999999998,5911]])
happy6 = np.array([[0,6783],[0.25,7065],[0.32249999999999995,7166],[0,7822],[0.8574999999999999,7918],[0,8285],[0,8575],[0.1525,9158],[0.0725,9432],[0,9575]],)
happy7 = np.array([[0,4850],[0.1175,5020],[0.02,5046],[0.0975,5046],[0.1475,5478],[0,5596],[0.305,5855],[0,5959],[0.02,6031],[0.25749999999999995,6031],[0,6149],[0.16,6221],[0.155,6501],[0,6660],[0.02,6767],[0.245,6767],[0.22749999999999998,7073],[0.02,7103],[0.04,7103]])
happy8 = np.array([[0,7594],[0,7887],[0.29,7970],[0,8275],[0.024999999999999998,8367],[0.565,8644],[0.1075,8734],[0,9161],[0.445,9436],[0.18,9514],[0.32,9811],[0.024999999999999998,9905],[0,10184],[0.23,10285]])
happy9 = np.array([[0,46656],[0.0975,46725],[0.08499999999999999,46840],[0,46961],[0.03,47090],[0.1775,47181],[0,47289],[0,47418]])
happy10 = np.array([[0,10389],[0.235,10627],[0.042499999999999996,11037],[0,11163]])
happy11 = np.array([[0,7810],[0,8610],[0,8707],[0,8807],[0,8906],[0,8997],[0,9089]])
happy12 = np.array([[0.10999999999999999,6407],[0,6555],[0,6655],[0,6694],[0.0225,6768],[0,6851],[0.2975,6916],[0,7000],[0,7077],[0.049999999999999996,7138],[0.1925,7266],[0.05499999999999999,7400]])
happy13 = np.array([[0,6233],[0.042499999999999996,6517],[0,6955],[0,7091],[0.12,7577],[0,7716]])
happy14 = np.array([[0,765423],[0,765608],[0.32749999999999996,765701],[0,765886],[0,766190],[0.245,766395],[0.045,766513],[0,766630],[0.1725,766920],[0,767186],[0.1575,767296],[0.0625,767394],[0.2075,767670],[0.21749999999999997,767979],[0.1775,768066],[0,768198],[0.19999999999999998,768436],[0,768687],[0,768862],[0.27249999999999996,768948],[0,769196],[0.345,769467],[0.32,769567],[0.08,769645],[0.375,769909],[0,771158],[0.02,771203],[0,771203],[0,771366],[0.315,771552],[0.1425,771752]])
happy15 = np.array([[0,10425],[0,10789],[0.355,11166],[0.4075,11516],[0.3125,11866],[0,12206],[0,12541],[0,12915],[0,13150],[0.10499999999999998,13283],[0,13617],[0,14025],[0,14183],[0,14341],[0.08499999999999999,14675],[0.0925,15075],[0,15226],[0,15393],[0.22749999999999998,15750]])
happy16 = np.array([[0,4786],[0.0025,5027],[0.05249999999999999,5211],[0,5360],[0.027499999999999997,5527],[0,5685],[0.015,5794],[0,5917],[0,6036],[0,6178],[0,6301],[0,6401],[0.11249999999999999,6528],[0,6727],[0.045,6787],[0,6895],[0.02,6964],[0.0825,7079],[0,7245],[0.02,7320],[0,7320],[0.04,7445],[0.03,7529],[0,8372],[0,8545],[0.02,8622],[0.0675,8622],[0,8720],[0.51,8803],[0,9755],[0.08,10065],[0.1075,10155],[0.0925,10246],[0,10363],[0.06999999999999999,10456],[0.0725,10579],[0.045,10665]],)
happy17 = np.array([[0,6209],[0.0675,6830],[0.215,7613],[0.135,8297],[0.18,8989],[0,9302],[0.1175,9589],[0.0575,9927],[0.2025,10297],[0.1675,10602],[0,10956],[0.0225,11235],[0.1575,11622],[0.005,11804],[0.18249999999999997,11997],[0.4325,12154],[0.0625,12362],[0.51,12541],[0.065,12680]])
happy18 = np.array([[0,7026],[0.02,7086],[0.2975,7086],[0,7242],[0,7575],[0.4075,7586],[0,7586],[0.17500000000000002,7768],[0,8242],[0.1525,8351],[0,8476],[0.09,8553],[0,9002],[0.8775,9089],[0,9217],[0.18249999999999997,9319],[0.13249999999999998,9848],[0,10045],[0.02,10162],[0.075,10162],[0,10646],[0.06999999999999999,10719],[0.1075,10865],[0.1875,10980],[0.04,11398],[0,11743],[0.10499999999999998,12010],[1,12271]])
happy19 = np.array([[0,165416],[0.01,165583],[0.1675,165824],[0.1275,166192],[0.10499999999999998,166516],[0,166770],[0,167248],[0.13749999999999998,167709],[0,167850],[0.1025,168201],[0.11249999999999999,168541],[0,168917],[0.18,169192],[0.0075,169412],[0,169668],[0,170057],[0.015,170344],[0.18,170672],[0,171002],[0,171292],[0.0225,171729],[0.15,172019],[0.0975,172325],[0,172729],[0,172980],[0.10499999999999998,173267],[0.0725,173454],[0.1675,173703],[0.16999999999999998,174061],[0,174397],[0.08,174723],[0,174861],[0.1175,175038],[0.19749999999999998,175288],[0.115,175632],[0,175963],[0.17500000000000002,176304]])
happy20 = np.array([[0,10627],[0,11026],[0,11444],[0,11781],[0,12053],[0,12385],[0.135,12580],[0.0025,12776],[0,13118],[0.19749999999999998,13417],[0,13664],[0.2075,13819],[0.01,14221],[0,14620],[0.1675,15071],[0.195,15510],[0.0975,15767],[0,15920],[0.13749999999999998,16321],[0,16537],[0.01,16899],[0,17019],[0,17312],[0.024999999999999998,17725]])
happy21 = np.array([[0,2859],[0,3275],[0.155,3540],[0,3734],[0,4145],[0,4542],[0,4917],[0.047499999999999994,5361],[0.22749999999999998,5692],[0.23,6058],[0,6332],[0,6785],[0.39,7239],[0,7626],[0.21999999999999997,8095],[0.027499999999999997,8496],[0.1225,8953],[0.2125,9120],[0.185,9345],[0.22749999999999998,9771]])
happy22 = np.array([[0.015,2089],[0.1725,2463],[0,2614],[0,2871],[0.0725,3230],[0,3632],[0.1875,3826],[0.20999999999999996,4040],[0.23,4424],[0.16999999999999998,4812],[0.02,5005],[0.20999999999999996,5201],[0,5582],[0.1875,5998],[0,6342],[0.1225,6704]])
happy23 = np.array([[0,2064],[0.075,2215],[0.24,2471],[0,2799],[0,2923],[0.2675,3205],[0,3549],[0,3821],[0.1075,4308],[0,4607],[0,4998],[0.049999999999999996,5196],[0,5327],[0.19749999999999998,5732],[0.13249999999999998,6052],[0,6251],[0,6417],[0,6689],[0.21999999999999997,6992],[0.1575,7333],[0.015,7625],[0.3,8026],[0,8186],[0.1925,8417]])
happy24 = np.array([[0,49175],[0.8225,49261],[0.2525,49261],[0.8999999999999999,49416],[0,49416],[0.045,49574],[0,49574],[0,51143],[0.625,51195],[0.0775,51195],[0.585,51375],[0,51375]])
happy25 = np.array([[0,12404],[0.012499999999999999,12573],[0,12660],[0.3,12750],[0.012499999999999999,12923],[0,13166]])
happy26 = np.array([[0,6136],[0.045,6261],[0.10999999999999999,6304],[0.13749999999999998,6304],[0,6454],[0.045,6488],[0.0075,6488],[0,6569],[0,6613],[0,6613],[0,8063],[0,8066],[0,8066],[0,8246],[0.0575,8454]])
happy27 = np.array([[0,2326],[0.042499999999999996,2896],[0.32749999999999996,3013],[0.024999999999999998,3479],[0.02,3551],[0.16,3551],[0,4303],[0.02,4391],[0,4391]])
happy28 = np.array([[0,4695],[0.09999999999999999,4794],[0.10499999999999998,4794],[0.027499999999999997,4839],[0.065,4839],[0.034999999999999996,4839],[0.08,4992],[0.1925,4992],[0.0075,7553]])
happy29 = np.array([[0.08,104769],[0,104812],[0,104898],[0,105230],[0.9899999999999999,105395],[0,105722],[1,105787],[0,105891],[0,106067],[0.825,106111],[0.6074999999999999,106111],[0,106224],[1,106251],[1,106251],[0,106580],[0,106733],[0.47,106810],[0.64,106904],[0,107017],[0,107133],[0.51,107251],[0,107383],[0,107484],[0,107593],[1,107658],[1,107751],[0,108397],[0.925,108457],[0.43499999999999994,108543],[0,108634],[0,108941],[0.02,108975],[0.38249999999999995,108975],[0,109110],[0,109391],[0,109467],[0,109611],[0.6475,109904],[0.6825,109993],[0,110126],[0,110260],[0,110613],[0.885,110620],[0.042499999999999996,110620],[0,111074],[0.18249999999999997,111085],[0.58,111085],[0,111453],[0,111526],[1,111670],[0,111964],[0,112069],[0,112169],[1,112252],[0,112329],[0.30749999999999994,112329],[1,112536],[0,112901],[1,112966],[0,113062],[0,113070],[1,113070],[1,113247],[1,113330],[0.9975,113362],[1,113362],[0,113681]])
happy30 = np.array([[0,4925],[0,5090],[0,5210],[0.21749999999999997,5526],[0.042499999999999996,5706],[0.115,5783],[0.02,5844],[0.25749999999999995,5844],[0.005,5923],[0.02,5989],[0.22749999999999998,5989],[0.02,6073],[0,6073],[0.02,6147],[0.2675,6147],[0,6430],[0.23249999999999998,6465],[0.02,6528],[0.034999999999999996,6528],[0.02,6618],[0.0725,6618],[0.024999999999999998,6704],[0.23,7143],[0.06,7377],[0,7608],[0.31,7674],[0.185,7774],[0,7856],[0,8217],[0,8811],[0,9302],[0.1075,9324],[0,9324],[0.0325,9678],[0,9678],[0.09,9772],[0.02,9813],[0,9813],[0,10079],[0.0575,10105],[0.03,10105],[0.02,10127],[0.075,10127],[0.1075,10127],[0.17500000000000002,10518],[0,10518],[0.02,10855],[0.015,10855],[0,11130],[0.0675,11207],[0.1225,11207],[0.02,11279],[0.0675,11279],[0,11594],[0.08499999999999999,11864],[0,12102],[0.09999999999999999,12110],[0.06999999999999999,12110],[0.09999999999999999,12175],[0.145,12175],[0.015,12175],[0.045,12197],[0.2525,12197],[0.09999999999999999,12197],[0.245,12197],[0,12546],[0,12694],[0,13085],[0,13215],[0,13509],[0.2125,13788],[0.047499999999999994,14551],[0.1725,14572],[0.22499999999999998,14572],[0.09,14580],[0.15,14580],[0.042499999999999996,14580],[0.235,14594],[0,14594],[0.13249999999999998,14896],[0,14896],[0.0975,15417],[0.0825,15417],[0,15633],[0.2675,15648],[0.37,15648],[0.475,15685],[0.37749999999999995,15685],[0.1575,15685],[0.024999999999999998,16580],[0.2075,16938],[0.255,17023]])
happy31 = np.array([[0,3918],[0.18999999999999997,4370],[0.58,4455],[0,4560],[0.85,4667],[0,5102],[0.045,5387],[0.13249999999999998,5645],[0.21749999999999997,5894],[0.16,6181],[0,6424],[0.185,6812],[0.16249999999999998,6969],[0.7525,7085],[0.23249999999999998,7464],[0.05499999999999999,7537],[0.27999999999999997,7865],[0.25749999999999995,8206],[0.02,8232],[0,8232],[0.46,8666],[0,9124],[0.045,9496],[0.575,9617],[0,9702],[0.725,9971],[0.09499999999999999,10213],[0,10571],[0.0925,10642],[0,11128],[0,11406],[0,11497],[0.027499999999999997,11514],[0,11514],[0.75,11603],[0.13,11746],[0.042499999999999996,11764],[0.042499999999999996,11764],[0,12031],[0,12113],[0,12113],[0.20999999999999996,12455],[0.02,12476],[0.042499999999999996,12476],[0,12715],[0.25749999999999995,12768],[0.2425,12785],[0.0975,12785],[0.8175,13304],[0.047499999999999994,13442],[0.06999999999999999,13673],[0.34249999999999997,13775],[0.2675,13871],[0.18249999999999997,13985],[0.2425,14198],[0.045,14288],[0,14482],[0,14750],[0.08,15160],[0,15454],[0.065,15943],[0,16234],[0,16770],[0.205,17477],[0.1525,17651],[0,17892],[0.36499999999999994,18240],[0,18559],[0,18748],[0,18849]])
happy32 = np.array([[0,7216],[0.3,7313],[0.06,7382],[0,7382],[0.23249999999999998,7726],[0.09999999999999999,7991],[0,7991],[0.075,8010],[0.012499999999999999,8010],[0.19999999999999998,8010],[0.16249999999999998,8184],[0.16249999999999998,8188],[0.012499999999999999,8188],[0.0925,8208],[0.1275,8208],[0,8208],[0.09999999999999999,8484],[0.2925,8562],[0.23249999999999998,8586],[0.09999999999999999,8586],[0,8768],[0.012499999999999999,8866],[0.0725,8876],[0.1025,8876],[0,9132],[0.235,9418],[0,9659],[0.1225,9902],[0.09999999999999999,9902],[0.042499999999999996,10401],[0,10613],[0.065,10800],[0.047499999999999994,10800],[0.02,10844],[0.0325,10844],[0.0975,10844],[0.2225,11287],[0.3175,11526],[0,11782],[0.115,11809],[0.10499999999999998,11809],[0.027499999999999997,11867],[0.8475,11867],[0.26499999999999996,11962],[0,11962],[0.0625,12366],[0.1475,12376],[0.05249999999999999,12376],[0.0925,12660],[0.2525,12674],[0.0625,12674],[0.52,12937],[0.09999999999999999,13086],[0.1025,13105],[0.125,13105],[0.02,13253],[0.02,13253],[0.1225,13253],[0.02,13259],[1,13259],[0,13620],[0.18,13630],[0.115,13630],[0.36499999999999994,14085],[0.6749999999999999,14317],[0,14528],[0,14802],[0.26499999999999996,15009],[0.0625,15264],[0,15366],[0.13999999999999999,15386],[0.16,15386],[0,15688],[0.135,15716],[0.5425,15716],[0.25,15799],[0.18,15885],[0,15885],[0.19999999999999998,16071],[0,16282],[0,16531],[0,16730],[0.235,16803],[0.01,16803],[0.027499999999999997,17105],[0.08,17269],[0,17506],[0,17718],[0,17927],[0.027499999999999997,18138],[0,18353],[1,18537]])
happy33 = np.array([[0,2375],[0.017499999999999998,2995],[0.185,3201],[0,3351],[0,3627],[0.11249999999999999,3700],[0,3816],[0,3924],[0,4176],[0.19749999999999998,4351],[0.02,4601],[0.22499999999999998,4601],[0,5057],[0.13249999999999998,5166],[0.0725,5265],[0.13749999999999998,5533],[0,5756],[0,6217],[0.15,6511],[0,6727],[0.185,6992],[0,6992],[0,7214],[0.13249999999999998,7689],[0.13749999999999998,7783],[0.027499999999999997,7884],[0.0625,8031],[0,8318],[0,8554],[0,8776],[0,9002],[0,9083],[0,9307],[0.0075,9538],[0.2125,9675],[0.2075,9762],[0.1275,9984],[0,10277],[0.23249999999999998,10438],[0.02,10505],[0.21999999999999997,10505],[0.21749999999999997,10752],[0,10868],[0,10972],[0,10972],[0.155,11093],[0.08499999999999999,11204],[0,11329],[0.19999999999999998,11420],[0.2225,11712],[0,11924],[0,12142],[0.3,12370],[0,12492],[0,12707],[0.20999999999999996,12836],[0.0225,13039],[0.08,13147],[0,13293],[0.10999999999999999,13387],[0,13526],[0.5525,13622],[0.2525,13622],[0,13876],[0,14003],[0,14177],[0.4875,14248],[0,14248],[0,14720],[1,14948],[0,15231],[0.6149999999999999,15238],[0,15238],[0,15713],[0,15922],[0,16147],[0.19999999999999998,16373],[0.6924999999999999,16589],[0.2125,16822],[0.9675,16914],[0.0325,17022],[0.06,17198],[0.26499999999999996,17265],[0,17690],[0.7525,18004]])
happy34 = np.array([[0,129656],[0.08750000000000001,129920],[0,130218],[0,130362],[0,130628],[0.045,130652],[0.1725,130652],[0,130877],[0.1475,131144],[0.09499999999999999,131471],[0,131736],[0.1275,131903],[0.0575,132004],[0.29,132004],[0.0375,132181],[0,132345],[0.1225,132550],[0.18249999999999997,132809],[0,133115],[0,133423],[0.1075,133433],[0.09999999999999999,133433],[0.06,133632],[0.1225,133876],[0.1775,133876],[0.06,134052],[0.08,134276],[0.0625,134632],[0.024999999999999998,134791],[0.1875,134791],[0,135069],[0,135308],[0.065,135342],[0.042499999999999996,135342],[0,135729],[0,135936],[0,136321],[0.08,136392],[0,136392],[0,136747],[0,136962],[0.0775,137382],[0.045,137432],[0.0675,137432],[0.13249999999999998,137746],[0.0725,137767],[0,137767],[0.08,138013],[0.08750000000000001,138440],[0.049999999999999996,138440],[0,138813],[0.03,138871],[0.06999999999999999,138871],[0.02,138963],[0.02,138963],[0.0075,138963],[0.05249999999999999,139331],[0.024999999999999998,139401],[0,139401],[0.024999999999999998,139459],[0.0375,139459],[0.03,139459],[0.02,139492],[0.027499999999999997,139492],[0.065,139492],[0,139492],[0.145,139924],[0.10999999999999999,139977],[0,139977],[0.11249999999999999,139989],[0,139989],[0,139989],[0.03,140418],[0.04,140442],[0,140442],[0.04,140491],[0,140491],[0.01,140491],[0.02,140536],[0,140536],[0.09999999999999999,140536],[0.0975,140536],[0,140897],[0.042499999999999996,140915],[0.065,140915],[0.02,140999],[0.0575,140999]])
happy35 = np.array([[0.0825,4691],[0.2125,5112],[0,5472],[0.18,5792],[0,6040],[0.115,6295],[0,6615],[0,6959],[0.11249999999999999,7171],[0.034999999999999996,7171],[0.0975,7181],[0.034999999999999996,7181],[0.047499999999999994,7181],[0.13749999999999998,7514],[0,7757],[0.02,7945],[0.047499999999999994,7945],[0.015,8224],[0.1025,8224],[0.024999999999999998,8510],[0.05499999999999999,8510],[0,8841],[0.02,8998],[0.205,8998],[0.1875,9273],[0.02,9464],[0.08750000000000001,9464],[0,9749],[0.0225,10080],[0,10325],[0.065,10565],[0.11249999999999999,10917],[0.06999999999999999,10934],[0.027499999999999997,10934],[0.015,11404],[0.1675,11535]])
happy36 = np.array([[0,1091],[0,1302],[0.1075,1645],[0.0325,1884],[0,2186],[0,2437],[0,2715],[0,3052],[0.2425,3316],[0.02,3454],[0.034999999999999996,3454],[0.0975,3774],[0.0375,3924],[0.0925,3924],[0.2525,4324],[0.045,4693],[0.2875,4693],[0,5039],[0.0725,5256],[0.017499999999999998,5256],[0.23,5633],[0.02,5766],[0.0025,5766],[0.18999999999999997,6084],[0.1075,6107],[0.042499999999999996,6107]])
happy37 = np.array([[0,1453],[0.2075,1926],[0.10499999999999998,2299],[0,2560],[0.1525,2801],[0,3111],[0.02,3262],[0.2925,3262],[0.1675,3684],[0.0375,3769],[0.03,3769],[0.11249999999999999,4230],[0.09499999999999999,4332],[0.0675,4332],[0,4682],[0.02,4844],[0.0825,4844],[0.1475,5227],[0.0625,5338],[0.2075,5338],[0.1075,5654],[0.024999999999999998,5799],[0.0025,5799],[0.1025,7994],[0.1875,8301],[0,8504],[0,8806]],)
happy38 = np.array([[0,2396],[0,2740],[0.04,2881],[0,3220],[0.0575,3575],[0.0725,3880],[0.12,4199],[0.06999999999999999,4368],[0.13999999999999999,4574],[0.04,4706],[0.0575,4706],[0,4891],[0.19749999999999998,5099],[0,5375],[0,5560]])
happy39 = np.array([[0,11892],[0.0775,12247],[0,12630],[0.0375,12971],[0.0375,13168],[0.02,13200],[0.3575,13200],[0,13583],[0,13739],[0.5075,14004],[0,14168],[0,14465],[0.135,14631],[0,14905],[0.13,15086],[0,15380],[0.0025,15540],[0.05249999999999999,15847],[0,16006],[0.0575,16259],[0.0625,16431],[0.09,16690],[0,16866],[0,17108],[0,17284],[0,17524],[0.0725,17643],[0.005,17936],[0.0675,18109],[0.015,18354],[0.0325,18490]])
happy40 = np.array([[0,3160],[0.2525,3427],[0,3635],[0.18999999999999997,3912],[0.11249999999999999,4142],[0,4385],[0,4545],[0.1575,4845],[0,4994],[0,5292],[0,5452],[0.1525,5720],[0.0925,5861],[0,6131],[0,6302],[0.1575,6570],[0.06,6694],[0,6954],[0,7136],[0,7385],[0,7529],[0,7735],[0,7924],[0.145,8235],[0.042499999999999996,8345],[0.09999999999999999,8662],[0,8786],[0,8986],[0.065,9169],[0.1575,9505],[0.015,9611],[0.08499999999999999,9952],[0,10081]])
happy41 = np.array([[0,1823],[0.02,1874],[0.17500000000000002,1874],[0.345,2215],[0.045,2331],[0.155,2331],[0,2648],[0.13,2820],[0.2825,3155],[0.045,3263],[0.047499999999999994,3263],[0,3597],[0,3764],[0.32249999999999995,4084],[0.13999999999999999,4165],[0.09999999999999999,4165],[0,4533],[0.08499999999999999,4629],[0.145,4629],[0.125,4999],[0.30749999999999994,5112],[0.125,5409],[0,5548],[0,5840],[0.10999999999999999,5966],[0.1225,6259],[0.12,6376],[0,6643],[0.16,6779],[0.1275,7070],[0,7182],[0,7496],[0,7623],[0.13249999999999998,7955],[0.1075,8080]])
happy42 = np.array([[0,1072],[0.11249999999999999,1172],[0.16249999999999998,1172],[0.3625,1540],[0.09,1654],[0.185,1654],[0.18,1964],[0.0925,2080],[0.1775,2080],[0.255,2440],[0.06,2556],[0.1025,2556],[0,2865],[0.04,2964],[0,2964],[0.2125,3309],[0.05249999999999999,3415],[0,3415],[0.2025,3716],[0.125,3858],[0.185,4170],[0,4280],[0.24,4546],[0.08,4656],[0.012499999999999999,4656],[0.2675,5016],[0.09999999999999999,5121],[0.10999999999999999,5465],[0.12,5577],[0,5875],[0.145,6006],[0,6306],[0.03,6438],[0.09,6438],[0.06,6744],[0.024999999999999998,6858],[0,6858],[0.1225,7176],[0.1575,7310],[0.22499999999999998,7576],[0.0625,7700],[0.042499999999999996,7700]])
happy43 = np.array([[0,1615],[0,1926],[0.06999999999999999,2057],[0.33999999999999997,2333],[0.042499999999999996,2519],[0.10499999999999998,2895],[0.02,3016],[0.2525,3016],[0.16999999999999998,3369],[0.13749999999999998,3692],[0,3927],[0.0025,3947],[0.145,3947],[0.045,4402],[0,4667],[0.06,4934],[0,5267],[0.17500000000000002,5584],[0.16999999999999998,5729],[0.1875,6045],[0.13,6294],[0.26,6501],[0.02,6580],[0.03,6580],[0,7044],[0.1025,7306],[0.13749999999999998,7762],[0,7969]])
happy44 = np.array([[0,4192],[0,4452],[1,4657],[0.5275,4799],[0.45249999999999996,4933],[0.335,5643],[0,5827],[0,6653],[0.5575,6791],[0,6856]])
happy45 = np.array([[0,1411],[0,1741],[0,2357],[0,2487],[0,2664],[0.2775,3252],[0,3374],[0,3483],[0,3733],[0.16249999999999998,4199],[0.17500000000000002,4341],[0,4510],[0,5201]])
happy46 = np.array([[0,4670],[0,5796],[0.315,6456],[0.18,6569],[0,6747],[0,6858],[0,7371],[0.13749999999999998,7907],[0,8449],[0.295,8604],[0.245,8759],[0.335,8857],[0.39249999999999996,9407],[0,10499],[0.18,10849]])
happy47 = np.array([[0,2699],[0.25,3126],[0,3247],[0.0975,3384],[0,3566],[0.2675,4009],[0.13249999999999998,4436],[0.12,4640],[0.1275,4839],[0.215,5022],[0,5181],[0,5602],[0,5718],[0,6044],[0.03,6212],[0.335,6511],[0.09499999999999999,6809]])
happy48 = np.array([[0,14760],[0,14971],[0.1025,15062],[0.195,15331],[0,15472],[0,15772],[0.1475,16012],[0.18249999999999997,16106],[0,16218],[0,16364],[0.25,16497],[0.0375,16873],[0.22749999999999998,17339]])
happy49 = np.array([[0,4304],[0.02,4343],[0.145,4343],[0.1525,4358],[0.042499999999999996,4358],[0.26499999999999996,4429],[0.02,4487],[0,4487],[0.125,4525],[0.1725,4544],[0.3175,4544],[0.06,4593],[0.145,4611],[0.034999999999999996,4611],[0.235,4695],[0.25749999999999995,4701],[0.3175,4701],[0.0075,4777],[0.185,4795],[0.19749999999999998,4795],[0.02,4844],[0.235,4844],[0.5675,4860],[0.2625,4860],[0.26,4936],[0.0325,4971],[0.012499999999999999,4971]])
happy50 = np.array([[0,6914],[0.3025,6925],[0.1275,6925],[0.305,6930],[0.1875,6930],[0,6930],[0.25749999999999995,6940],[0.1225,6940],[0.1725,6940],[0,7063],[0.1925,7070],[0.125,7070],[0.0375,7116],[0.0575,7123],[0.215,7123],[0.02,7221],[0.18249999999999997,7221],[0.145,7221]])


sad1 = np.array([[0,5903],[0.02,6011],[0,6011],[0,7737],[0.02,7821],[0.16999999999999998,7821]])
sad2 = np.array([[0,3236],[0,4795],[0,6681],[0.1775,8872]])
sad3 = np.array([[0,6397],[0.0075,7940],[0.0025,8312],[0,9639],[0,10158],[0,10442]])
sad4 = np.array([[0,4082],[0.017499999999999998,5892],[0,8800],[0,8984]])
sad6 = np.array([[0,3308],[0,3308],[0.02,4340],[0.027499999999999997,4340],[0.02,5431],[0.08,5431],[0.02,6526],[0.1225,6526],[0.02,7733],[0.43999999999999995,7733],[0.02,9332],[0.017499999999999998,9332]])
sad7 = np.array([[0,1193],[0,2403],[0.09,4149],[0,5593]])
sad8 = np.array([[0,1193],[0,2403],[0.09,4149],[0,5593]])
sad9 = np.array([[0,1619],[0,3336],[0,5457],[0.034999999999999996,7058],[0.04,7058]])
sad10 = np.array([[0,19868],[0,22301],[0,24652],[0.02,26736]])
sad11 = np.array([[0,15804],[0,16929],[0,18094],[0.0025,18709],[0.16999999999999998,19387],[0,20168],[0.047499999999999994,20387],[0.03,20670],[0.165,21287],[0,22145]])
sad12 = np.array([[0.165,2465],[0,3689],[0,4206],[0,5055],[0,6109],[0.08750000000000001,6587],[0.0225,7434],[0.03,8493]])
sad13 = np.array([[0,6343],[0.065,7381],[0,7582],[0,8196],[0.0075,9066],[0,9602],[0,10290],[0,11060],[0,11352],[0,11657]])
sad14 = np.array([[0,3042],[0,4109],[0,4900],[0,6051],[0.04,6878],[0,7745]])
sad15 = np.array([[0,3639],[0.05249999999999999,3945],[0,4871],[0.0025,5400],[0,6117],[0,6552],[0,6812],[0.015,7560],[0.0375,8203],[0,9193]])
sad16 = np.array([[0,3358],[0.0975,3821],[0,4393],[0,4897],[0,5375]])
sad17 = np.array([[0,3033],[0,3678],[0.0325,4263],[0,5007],[0,5728],[0,6478]])
sad18 = np.array([[0,9907],[0,10012],[0,10117],[0,10260],[0.0075,10374],[0,10481],[0,10640],[0.0325,10663],[0.005,10663],[0,10920],[0,10934],[0.0075,10934],[0,11179],[0,11379]])
sad19 = np.array([[0,5959],[0,6157],[0,6515],[0,6833],[0.18249999999999997,6850],[0,6850],[0,7227],[0.09499999999999999,7254],[0,7254],[0,7414],[0.06,7436],[0,7436],[0,7643],[0,7655],[0,7655],[0,7816],[0.027499999999999997,7864],[0,7864],[0,8036],[0,8201],[0.03,8218],[0,8218]])
sad20 = np.array([[0,5881],[0.0025,6078],[0.047499999999999994,6270],[0.065,6426],[0.05499999999999999,6575],[0.049999999999999996,6727],[0.024999999999999998,6883],[0,7041],[0,7362],[0,7501],[0,7657]])
sad21 = np.array([[0,4402],[0.049999999999999996,4770],[0,4770],[0.06,5010],[0,5010],[0,5010],[0.13,5281],[0.02,5408],[0.19999999999999998,5544],[0.155,5864],[0.0725,6112],[0.024999999999999998,6112],[0.0225,6378],[0,7030],[0.02,7939],[0.0225,7939],[0.03,8145],[0.03,8315],[0.0075,8427],[0.115,8664],[0.0725,8805],[0.049999999999999996,8805],[0.05499999999999999,9110],[0.02,9110],[0,9110],[0.13999999999999999,9480],[0,9773],[0.02,9795],[0,9795],[0.04,10003],[0,10143],[0,10252],[0,10425],[0.042499999999999996,10520],[0,10692],[0.17500000000000002,10793],[0,11085],[0.045,11330],[0.135,11330],[0.0225,11576],[0.02,11576],[0.01,11576],[0,11850],[0.0625,11925],[0.02,12055],[0.1425,12055],[0.02,12585],[0.1225,12606],[0.13999999999999999,12606],[0,12850],[0.09,12871],[0,12871],[0.0225,12882],[0,12882],[0.1475,12882],[0.017499999999999998,13111],[0.08,13111],[0.017499999999999998,13123],[0.06,13123],[0.0625,13123],[0.075,13367],[0.0725,13391],[0.10499999999999998,13391],[0,13391],[0.03,13594],[0.05249999999999999,13615],[0.09,13615],[0,13615],[0,13849],[0.0675,13863],[0,13863],[0.1275,13863],[0,14095],[0.08,14118],[0,14118],[0.13749999999999998,14118],[0.1225,14359],[0.0925,14376],[0.09499999999999999,14376],[0.0575,14379],[0.06,14379],[0,14379],[0,14587],[0,14605],[0.0925,14605],[0,14614],[0.04,14614],[0.1675,14614],[0,14847],[0.065,14847],[0.05499999999999999,14868],[0.04,14868],[0.0725,14868],[0.065,16414],[0.0925,16541],[0.02,16603],[0.005,16603],[0.02,16730],[0.049999999999999996,16730]])
sad22 = np.array([[0,1558],[0.02,2248],[0.01,2248],[0.08750000000000001,2400],[0.02,2528],[0.0975,2528],[0.02,2809],[0.19999999999999998,2809],[0.02,3102],[0.0075,3102],[0.1525,3264],[0.0025,3264],[0.02,3358],[0.049999999999999996,3358],[0,3358],[0.205,3652],[0.02,3902],[0.2625,3902],[0.02,4183],[0.255,4183],[0.1875,4305],[0.08499999999999999,4305],[0.0625,4377],[0.215,4377],[0.02,4683],[0.19749999999999998,4683],[0.2125,4962],[0.19999999999999998,5201],[0,5452],[0,5739],[0.135,6877],[0.03,7312],[0.02,7509],[0,7509],[0.0375,8465],[0.049999999999999996,8465],[0.05499999999999999,8531],[0.012499999999999999,8531],[0.05499999999999999,8682],[0.012499999999999999,8682],[0.017499999999999998,8682],[0.049999999999999996,8874],[0.02,8874],[0.005,8874],[0.065,9002],[0.01,9002],[0.012499999999999999,9002],[0.02,9239],[0.03,9239],[0.02,9239],[0.049999999999999996,9376],[0.1175,9376],[0.125,9467],[0.0925,9467],[0.04,9478],[0.10999999999999999,9478],[0,9478],[0.09,9565],[0.13,9565],[0.09,9659],[0.18249999999999997,9659],[0.585,9697],[0,9697],[0.02,9748],[0.11249999999999999,9748],[0.027499999999999997,9748],[0,9970],[0.16,10034],[0.04,10034],[0.02,10074],[0.09999999999999999,10074],[0.1525,10298],[0.02,10298],[0.1075,10759],[0.1025,10759],[0.0675,11148],[0.049999999999999996,11148],[0.02,11576],[0.1575,11576],[0,11576],[0.0575,11910],[0,11910],[0.01,11910],[0.02,12055],[0.155,12055],[0.02,12180],[0.0975,12180],[0.0225,12543],[0.1425,12543],[0.02,13352],[0.075,13352],[0.13999999999999999,13352]])
sad23 = np.array([[0,1137],[0.0375,1365],[0,1365],[0.0375,1416],[0.0325,1416],[0,1416],[0.0375,1954],[0.0325,1954],[0,1954],[0.02,2501],[0,2501],[0.02,2596],[0.0075,2596],[0.042499999999999996,2596],[0.0075,2788],[0.024999999999999998,2788],[0,2788],[0.0075,3287],[0.02,3287],[0.017499999999999998,3287],[0.017499999999999998,3478],[0,3478],[0.05249999999999999,3650],[0.0325,3650],[0.02,3921],[0.135,3921],[0.0775,4253],[0,4253],[0.0675,4571],[0.05249999999999999,4571],[0,4910],[0.0325,5199],[0.075,5353],[0.06999999999999999,5353],[0.0825,5420],[0.02,5420],[0.03,5420],[0.02,5696],[0.1025,5696],[0.13249999999999998,5766],[0.017499999999999998,5766],[0.135,5906],[0.017499999999999998,5906],[0,5906],[0.02,6739],[0,6739],[0.02,6972],[0.015,6972],[0.02,7266],[0.185,7266],[0.0975,7574],[0,7574],[0.015,7688],[0,7688],[0.02,7855],[0,7855],[0.0375,8803],[0,8803],[0.015,8981],[0.0575,9083],[0.0325,9101],[0.02,9115],[0.017499999999999998,9115],[0.02,9127],[0.027499999999999997,9247],[0.017499999999999998,9247],[0.0725,9256],[0.034999999999999996,9267],[0.017499999999999998,9267],[0.0775,9270],[0.04,9280],[0.017499999999999998,9280],[0.0825,9294],[0.045,9296],[0.0725,9331],[0.034999999999999996,9362],[0.024999999999999998,9364],[0.027499999999999997,9379],[0.02,9381],[0.02,9382],[0.02,9382],[0.02,9413],[0.02,9513],[0.05249999999999999,9513],[0.065,9555],[0,9555],[0.02,9686],[0.012499999999999999,9686],[0.03,9686],[0.02,9810],[0.0675,9810],[0.06,9845],[0.027499999999999997,9845],[0.024999999999999998,9921],[0,9921],[0.0325,10059],[0,10059],[0.0675,10356],[0,10356],[0.03,10674],[0,10674],[0.01,10918],[0.0075,10918],[0.012499999999999999,11269],[0.02,11269],[0.02,11269],[0.115,11632],[0.02,11750],[0.1225,11750],[0.02,11888],[0.042499999999999996,11888],[0.09,11956],[0.0725,11956],[0.02,12063],[0.02,12063],[0.0075,12063],[0.02,12407],[0.08499999999999999,12407],[0.049999999999999996,12508],[0.049999999999999996,12508],[0.02,12695],[0.02,12695],[0.02,12942],[0.005,12942],[0.02,13210],[0,13210],[0.042499999999999996,13386],[0.017499999999999998,13386],[0.042499999999999996,13451],[0.017499999999999998,13451],[0,13451],[0,14412],[0.02,14416],[0.0225,14416]])
sad24 = np.array([[0,2233],[0.0725,2413],[0,2413],[0.10999999999999999,2616],[0,2616],[0,2616],[0.049999999999999996,2847],[0.0025,2847],[0,2847],[0.0625,3067],[0,3067],[0,3067],[0,3281],[0.01,3281],[0,3569],[0.01,3569],[0.0225,3569],[0.01,4417],[0.115,4417],[0,4417],[0,4718],[0,4718],[0.0025,4965],[0.049999999999999996,4965],[0.049999999999999996,5142],[0,5142],[0.0625,5230],[0,5230],[0,5230],[0,5518],[0.005,5518],[0.049999999999999996,5547],[0,5547],[0.05249999999999999,5676],[0.015,5676],[0,5676],[0.005,6230],[0,6230],[0.005,6259],[0.04,6259],[0,6259],[0.0725,6462],[0,6462],[0,6462],[0.02,6727],[0,6727],[0.047499999999999994,6775],[0,6775],[0.0325,6921],[0,6921],[0,6921],[0.02,7075],[0,7075],[0.02,7255],[0.0375,7255],[0.02,7379],[0,7379],[0.0625,7493],[0,7493],[0.08499999999999999,7638],[0,7638],[0,7638],[0.0725,8161],[0,8161],[0.11249999999999999,8291],[0,8291],[0,8291],[0.012499999999999999,9105],[0,9105],[0.02,9665],[0,9665],[0.017499999999999998,10078],[0.0075,10078],[0.017499999999999998,10394],[0.05499999999999999,10394],[0,10394],[0.0625,10652],[0,10652],[0.01,10652],[0.01,10931],[0.005,10931],[0.01,11055],[0.0575,11055],[0,11055],[0.0825,11282],[0,11282],[0,11282],[0.0025,11483],[0,11483],[0.0025,11542],[0.0625,11542],[0,11542],[0.04,11723],[0.02,11723],[0,11723],[0,12271],[0.024999999999999998,12271],[0,12482],[0.049999999999999996,12482],[0,12482],[0.065,12697],[0.0025,12697],[0.012499999999999999,12697],[0.017499999999999998,12959],[0,12959],[0.02,13074],[0,13074],[0.02,13177],[0.017499999999999998,13177]])
sad25 = np.array([[0,2808],[0.02,4033],[0.12,4033],[0.034999999999999996,4260],[0.02,4265],[0.024999999999999998,4310],[0.09499999999999999,4448],[0.06999999999999999,4551],[0,4551],[0.024999999999999998,4584],[0.0225,4584],[0,4584],[0,4749],[0.045,4832],[0.01,4832],[0.02,4902],[0.02,4902],[0.08750000000000001,4902],[0.027499999999999997,5203],[0.0325,5203],[0.027499999999999997,5307],[0,5307],[0.0975,5424],[0.0975,5559],[0.0625,5692],[0.0625,5869],[0.02,6004],[0.0625,6004],[0.0225,6167],[0.03,6221],[0.04,6221],[0.02,6319],[0.02,6319],[0.0375,6319],[0.02,6408],[0,6408],[0.08,6520],[0.042499999999999996,6520],[0.02,6572],[0.02,6572],[0,6572],[0,6762],[0.024999999999999998,6806],[0.10999999999999999,6806],[0.0375,6915],[0.0375,6915],[0.0725,6915],[0.075,8364],[0.0375,8364],[0,8364],[0.045,8687],[0,8687],[0.02,8923],[0.045,8923],[0.13249999999999998,9112],[0,9112],[0.06999999999999999,9159],[0.03,9159],[0,9159],[0,9362],[0.065,9433],[0.065,9433],[0.02,9503],[0.02,9503],[0.0225,9503],[0.027499999999999997,9646],[0,9646],[0.02,10031],[0.05249999999999999,10031],[0.0825,10031],[0.005,10704],[0.005,10710],[0.22499999999999998,10710],[0.22749999999999998,12083],[0.02,12090],[0.06,12090],[0.0675,12090],[0,12333],[0.1525,12555],[0.0675,12832],[0.08,12832],[0.09499999999999999,13117],[0.02,13117],[0,13117],[0.01,13827],[0.06,13949],[0.02,14030],[0.0325,14030],[0.02,14263],[0,14263],[0.02,14286],[0.0625,14286],[0.075,14286]])
sad26 = np.array([[0,4691],[0,5900],[0.04,6625],[0.065,7199],[0,7670],[0,8159],[0,8575],[0,9002],[0,9408],[0.0825,9879],[0.0325,10235],[0,10644],[0,11162],[0.0625,11676]])
sad27 = np.array([[0,2077],[0.1075,2767],[0.13999999999999999,3159],[0.145,3591],[0.1025,4042],[0.21749999999999997,4445],[0,4774],[0.049999999999999996,5245],[0.075,5605],[0,6012],[0,6252]])
sad28 = np.array([[0.2375,1064],[0.25,1281],[0,1471],[0,1663],[0,1888],[0.13249999999999998,2414],[0.08750000000000001,2879],[0.0375,3305],[0.005,3773],[0.0325,4549],[0.10499999999999998,4864]])
sad29 = np.array([[0,2262],[0,2737],[0,3028],[0,3348],[0,3797],[0,4211],[0,4623],[0,4995],[0,5381],[0,5721],[0,6014],[0.0925,6348]])
sad30 = np.array([[0,821],[0,1261],[0.0325,1621],[0.09,1946],[0,2445],[0.06,2847],[0,3163],[0.09999999999999999,3630],[0.06999999999999999,3978],[0.0025,4355],[0,4756],[0.034999999999999996,5144]])
sad31 = np.array([[0,4023],[0.145,4524],[0,5045],[0,5699],[0,6298],[0,6856],[0.042499999999999996,7501],[0.042499999999999996,8125],[0,8926],[0.012499999999999999,9476],[0.1075,10141],[0.075,10676],[0,11333],[0.075,11953],[0.145,12561],[0.08750000000000001,13125]])
sad32 = np.array([[0.075,1277],[0.08,1929],[0,2604],[0,3337],[0.027499999999999997,4006],[0,4814],[0.10999999999999999,5564],[0.045,6314],[0,7123],[0.0725,7731],[0.09,8130],[0.05499999999999999,8999],[0.015,9648]])
sad33 = np.array([[0,1425],[0.1675,2076],[0.0725,2667],[0.012499999999999999,3358],[0.0675,3970],[0,4701],[0.047499999999999994,5437],[0,6129],[0.0075,6870],[0.024999999999999998,7570],[0.005,8254],[0.047499999999999994,8920],[0.0925,9589],[0,10245],[0.0025,10930],[0.0625,11564],[0,12204]])
sad34 = np.array([[0,1362],[0,2273],[0.034999999999999996,2630],[0,3431],[0.05499999999999999,4168],[0,4843],[0.0575,5485],[0,6344],[0,7052],[0,7708],[0,8383],[0.027499999999999997,9216],[0,9854],[0,10528],[0.02,11208],[0.01,11835],[0,12679],[0.11249999999999999,13378]])
sad35 = np.array([[0,1218],[0,1762],[0.0375,2471],[0,3170],[0,3863],[0.0675,4472],[0,5279],[0,5871],[0.08750000000000001,6625],[0.06999999999999999,7315],[0,7994],[0.0025,8692],[0.0225,9324],[0.0775,10057],[0.0375,10699],[0,11392]])
sad36 = np.array([[0,2718],[0.015,3393],[0.08750000000000001,4070],[0.13,4367],[0,5031],[0.09999999999999999,5684],[0,6403],[0.0825,7103],[0.18249999999999997,7837]])
sad37 = np.array([[0,1937],[0,2590],[0,3100],[0.0375,3613],[0,4031],[0.08750000000000001,4373],[0.042499999999999996,5100],[0,5565],[0.02,6170]])
sad38 = np.array([[0,986],[0,1758],[0,2059],[0,2609],[0,3470],[0.02,3835],[0,4244],[0.01,4763],[0.05499999999999999,5254]])
sad39 = np.array([[0,1543],[0.0575,2099],[0.0225,2799],[0,3165],[0,3558],[0.13,4185]])
sad40 = np.array([[0.0375,1413],[0.0625,2086],[0,2351],[0,3061],[0,3330],[0.005,3585],[0.0375,4204],[0,4637]])
sad41 = np.array([[0,1770],[0,1953],[0,1966],[0.03,1966],[0,2639],[0.1025,2727],[0,2727],[0,3597]])
sad42 = np.array([[0.0025,3043],[0.0925,3332],[0,3332],[0,4111],[0.065,4414],[0,4414],[0,5510]])
sad43 = np.array([[0,1345],[0.047499999999999994,1706],[0.1575,1706],[0,2545],[0.045,3309],[0.045,3309]])
sad44 = np.array([[0.0025,3204],[0.012499999999999999,3564],[0.012499999999999999,3564],[0,5628],[0.0075,6127],[0.0375,6127]])
sad45 = np.array([[0,4036],[0,4104],[0,4140],[0,4140],[0,5204],[0.0075,5746],[0,5746],[0,6460],[0,7788]])
sad46 = np.array([[0,3428],[0.03,3992],[0.1775,4672],[0,4955],[0.065,5638],[0,6208],[0.015,7004],[0,7304],[0.1175,7803],[0.16999999999999998,8234],[0.09999999999999999,8653],[0.024999999999999998,9105]])
sad47 = np.array([[0,2498],[0,2863],[0.2125,3141],[0,3641],[0,3932],[0,4220],[0.024999999999999998,4532],[0,4916],[0.215,5196],[0.017499999999999998,5714],[0,6236],[0.285,6450],[0,6868],[0,7218],[0.16999999999999998,7609],[0.2075,7909],[0,8166],[0,8567],[0,9010]])
sad48 = np.array([[0.18249999999999997,1314],[0,2005],[0,2373],[0.0725,2854],[0,3245],[0.115,3673],[0.0725,4010],[0.2375,4572],[0.1175,5382],[0.16999999999999998,6175],[0,6651],[0,6992],[0.02,7426],[0.0375,8259],[0.02,9068],[0.0325,9569],[0,9732]])
sad49 = np.array([[0,3293],[0.09499999999999999,3975],[0.042499999999999996,4669],[0,5066],[0.11249999999999999,5693],[0,6431],[0.05499999999999999,6694],[0,7082],[0,7597],[0,8002],[0.12,8694],[0.075,9470],[0.05499999999999999,9808],[0,10079],[0.027499999999999997,10550]])
sad50 = np.array([[0,1999],[0.017499999999999998,3049],[0,3612],[0.034999999999999996,3953],[0.12,4707],[0.09,5179],[0.034999999999999996,5976],[0.05249999999999999,6784],[0,7224],[0.0675,8232],[0.017499999999999998,9329],[0,9655],[0,10415],[0.047499999999999994,10898],[0,11825],[0.01,12167]])

angry1 = np.array([[0,94871],[0,94978],[0.2875,95052],[0,95173],[0.2525,95251],[0.425,95334],[0,95449],[0.02,95494],[0.7474999999999999,95494],[0,95821],[0.13749999999999998,95843],[0.065,95843],[0.255,95990],[0.6325,96018],[0.25749999999999995,96018]])
angry2 = np.array([[0,3887],[0,4063],[0.19749999999999998,4193],[0.7575,4442],[0.7649999999999999,4634],[0.36499999999999994,4836],[0.02,5027],[0.32499999999999996,5027],[0.39749999999999996,5272],[0.26,5454],[0.3,5635],[0,5860],[0.11249999999999999,6035],[0,6221],[0,6428],[0,6624]])
angry3 = np.array([[0.46499999999999997,3031],[0.02,3086],[0,3086],[0,3330],[0.165,3352],[0.1725,3352],[0.425,3622],[0.02,3687],[0.16999999999999998,3687],[0,3902],[0.0575,3963],[0,3963],[0,4286],[0,4517],[0.045,4539],[0.0925,4539],[0.075,4824],[0.02,4873],[0.012499999999999999,4873]])
angry4 = np.array([[0.1225,9526],[0.0075,9686],[0.05499999999999999,9769],[0.09499999999999999,9769],[0.1475,9938],[0.1075,10010],[0.05249999999999999,10010],[0.09499999999999999,10115],[0.02,10115],[0.06999999999999999,10377],[0.22499999999999998,10435],[0.08499999999999999,10435],[0.02,10573],[0.0975,10573],[0.09999999999999999,10703],[0.205,10787],[0,10787],[0.16999999999999998,11170],[0.1775,11285],[0.02,11367],[0,11367],[0,11525],[0.41999999999999993,11561],[0,11561],[0,11978],[0.1025,12275],[0.26,12275],[0.0225,22093]])
angry5 = np.array([[0,7162],[0.37749999999999995,7218],[0.115,7218],[0.005,7367],[0.3675,7443],[0,7443],[0.0775,7570],[0.145,7669],[0.0225,7750],[0.21999999999999997,7750],[0.13,7919],[0.2975,7925],[0.0075,7925],[0.3325,8135],[0.41999999999999993,8202],[0.0675,8202],[0.1175,8460],[0.045,8521],[0.05499999999999999,8521],[0.2775,8674],[0.2975,8699],[0.042499999999999996,8699],[0.2975,8866],[0.09,8866],[0.295,9031]])
angry6 = np.array([[0,11888],[0,12020],[0.09,12317],[0,12451],[0,12731],[0.0775,12877],[0.012499999999999999,13161],[0.3625,13289]])
angry7 = np.array([[0,10729],[0.042499999999999996,10870],[0,11128],[0,11249],[0,11472],[0,11590],[0,11806],[0,11914],[0,12131],[0,12239],[0.01,12466],[0,12574]])
angry8 = np.array([[0.0375,6483],[0,7495],[0.2075,7816],[0.1075,8112],[0.1275,8369],[0.565,8613]])
angry9 = np.array([[0,5279],[0.125,5423],[0.13,5808],[0,6323],[0,6481],[0,6767],[0.01,6910]])
angry10 = np.array([[0,2645],[0.215,2979],[0.1025,3563],[0,3703],[0,4180],[0,4319]])
angry11 = np.array([[0,8743],[0,10108],[0,11357],[0.13999999999999999,12423],[1,12423],[0,14064],[0,14064],[0,15074],[0,15074]])
angry12 = np.array([[0,3740],[0.02,3752],[0.505,3752],[0.315,4633],[0.02,4637],[0.39499999999999996,4637],[1,5562],[0,5562],[0,6580],[0.3525,6612],[0,6612],[0,7592],[0.85,7610],[0,7610],[0,8566],[0,8573],[0,8573]])
angry13 = np.array([[0,4592],[0.09999999999999999,5018],[0,5310],[0,5464],[0,5593],[0.7025,5718],[0,5867],[0.7674999999999998,5953],[0,6110],[0,6219],[0,6343],[0.725,6419],[0,6534],[0.02,6561],[0.66,6561],[0,6868],[0.06999999999999999,6886],[0,6886]])
angry14 = np.array([[0,12750],[0,12750],[0,13210],[0.1275,13214],[0.09499999999999999,13214],[0.18249999999999997,13626],[0.02,13632],[0.12,13632],[0.012499999999999999,14105],[0.0575,14105],[0.0375,14634],[0.18249999999999997,14634],[0.0025,15217],[0.7825,15235],[0.1575,15235],[0.1675,15377],[0,15377],[0.0675,15553],[0,15553]])
angry15 = np.array([[0,3201],[0,3201],[0.0575,3215],[0,3215],[0.7599999999999999,3215],[0.16249999999999998,3215],[0.155,3659],[0,3659],[0.39749999999999996,3659],[1,3670],[0,3670],[0.02,3670],[0,3670],[0.7649999999999999,4170],[0.02,4182],[1,4182],[0.0625,4182],[0,4182],[0.10999999999999999,4745],[0.5325,4745],[0.5975,4748],[0.02,4748],[0,4748],[0.02,4752],[0.335,4752],[0,4752],[0,4752],[1,5385],[0.4625,5404],[0.5175,5404],[0.24749999999999997,5416],[0.19749999999999998,5416],[0,5416],[0.06,5419],[0.37999999999999995,5419],[0,5419],[0.32,5419]])
angry16 = np.array([[0,22334],[0,22342],[0.59,22342],[0.19749999999999998,22520],[0,22701],[0.27999999999999997,22789],[0,22943],[0,23034],[0,23193],[0,23303],[0,23528],[0.1575,23568],[0,23568],[0,23793],[0,23976],[0.1925,23989],[0.0675,23989],[0.26,24194],[0.16,24392],[0,24582],[0.045,24917],[0.06999999999999999,25101],[0,25172],[0.0675,25370],[0,25452],[0,25452],[0.02,25464],[0.02,25464],[0.01,25464],[0,25555],[0,25658],[0.024999999999999998,25759],[0,25866],[0.0325,25960],[0,25960],[0.09,25975],[0,25975],[0.39499999999999996,25975],[0,26065],[0.02,26198],[0,26303],[0.042499999999999996,26522],[0,26732],[0.02,26759],[0.05499999999999999,26759],[0,26903]])
angry17 = np.array([[0,2927],[0,2958],[0,2958],[0,3104],[0,3164],[0.08499999999999999,3330],[0.18249999999999997,3330],[0.05249999999999999,3379],[0.25749999999999995,3564],[0,3656],[0.205,3863],[0,3991],[0.1475,4147],[0,4365],[0,4538],[0,4538],[0.215,4628],[0.23249999999999998,4848],[0.0225,4883],[0,4883],[0,5055],[0,5067],[0.32499999999999996,5067],[0.245,5178],[0,5415],[0.19999999999999998,5499],[0,5679],[0,5822],[0,6099],[0,6232],[0.10999999999999999,6616],[0.042499999999999996,6677],[0.09,6839],[0,6978],[0,7066]])
angry18 = np.array([[0,3891],[0.26,4082],[0.0575,4145],[0,4248],[0,4346],[0.125,4488],[0,4592],[0.2825,4720],[0.0375,4731],[0,4731],[0.16249999999999998,4740],[0.065,4740],[0,4740],[0.145,4924],[0,5291],[0.13749999999999998,5319],[0,5319],[0.04,5336],[0.02,5336],[0.065,5761],[0.015,5917],[0.0775,6063],[0.03,6063],[0,6316],[0,6316],[0,6475],[0,6612],[0.24749999999999997,6695],[0.0675,6704],[0.042499999999999996,6704],[0,6829],[0.0575,6860],[0.03,6860],[0.27999999999999997,6946],[0.1275,6961],[0.1725,6961],[0,7096],[0.0225,7208],[0.5075,7208],[0.1175,7408],[0.0225,7435],[0.024999999999999998,7435],[0.32749999999999996,7465],[0,7604],[0.05249999999999999,7683],[0.0025,7683],[0.1525,7832]])
angry19 = np.array([[0,3323],[0.0225,3352],[0.065,3352],[0.0625,3399],[0,3559],[0,3569],[0.4225,3569],[0,3618],[0,3751],[0,3774],[0.045,3774],[0.11249999999999999,3789],[0.065,3789],[0.21749999999999997,3821],[0.049999999999999996,3951],[0.02,3965],[0.49,3965],[0.3175,3965],[0,4156],[0.045,4375],[0,4460],[0.005,4475],[0.25,4475],[0.10999999999999999,4536],[0,4625],[0,4664],[0.04,4664],[0,4664],[0.04,4718],[0.09,4718],[0.0975,5111],[0.042499999999999996,5443],[0,5596],[0.05499999999999999,5696],[0,5804],[0,5921],[0,6038],[0.17500000000000002,6155],[0.215,6243],[0.135,6505],[0,6783],[0.06999999999999999,6802],[0.017499999999999998,6802],[0.16249999999999998,7018],[0.02,7057],[0.3325,7057],[0,7212]])
angry20 = np.array([[0,4218],[0.25749999999999995,4314],[0,4441],[0,4502],[0,4515],[0.2425,4515],[0,4639],[0,4715],[0,4834],[0,4920],[0,5019],[0.0825,5090],[0.165,5275],[0.16249999999999998,5453],[0,5577],[0,5661],[0,5750],[0,5861],[0.08499999999999999,5949],[0,6053],[0.005,6137],[0.01,6311],[0.13249999999999998,6338],[0,6472],[0,6590],[0.1025,6697],[0.03,6765],[0.0225,6866],[0,7059],[0.03,7089],[0.0925,7089],[0,7202],[0.2975,7396],[0,7396],[0.005,7494],[0.155,7652],[0.1875,7729],[0,7828],[0.02,7861],[0.09999999999999999,7861],[0,8044],[0.02,8084],[0.1525,8084],[0,8244],[0.047499999999999994,8270],[0.1875,8270],[0.01,8378],[0,8520],[0,8607],[0.08499999999999999,8746],[0.0825,8778],[0.09999999999999999,8778],[0.10499999999999998,9012],[0.02,9035],[0.245,9035],[0.25,9228]])
angry21 = np.array([[0,11349],[0,11362],[0,11362],[0,11362],[0,11364],[0,11364],[0.03,11364],[0.2125,11364],[0,11584],[0,11584],[0.25,11817],[0.25749999999999995,11817],[0,12002],[0,12002],[0,12025],[0.02,12025],[0.385,12025],[0,12025],[0,12233],[0,12246],[1,12246],[0.32249999999999995,12455],[0.024999999999999998,12455],[0.02,12476],[0.05249999999999999,12476],[0.8574999999999999,12476],[0,12476],[0.32499999999999996,12664],[0.1725,12664],[0.02,12686],[0.02,12686],[0,12686],[0.4825,12686],[0.66,12914],[0.042499999999999996,12914],[0,12914]])
angry22 = np.array([[0.0825,2692],[0.43,2907],[0.305,3113],[0.1925,3319],[0.49249999999999994,3543],[0.29,3761],[0.4225,3987],[0.7274999999999999,4187],[0.024999999999999998,4385]])
angry23 = np.array([[0,5698],[0.02,5713],[0,5713],[0.02,5722],[0,5722],[0.27999999999999997,5722],[0.4725,5911],[0.02,5919],[0,5919],[0.02,5924],[0,5924],[0,5924],[0,6131],[0.41,6131],[0,6141],[1,6141],[0.71,6141],[0,6373],[0.32749999999999996,6373],[0.0075,6609],[0.36499999999999994,6609],[0.0075,6620],[1,6620],[0.8125,6620],[0.43999999999999995,6859],[1,6866],[1,6866],[0,6866]])
angry24 = np.array([[0,3857],[0.1225,4068],[0,4258],[0.06,4299],[0.0575,4299],[0.0725,4430],[0.49499999999999994,4472],[0.2525,4472],[0.45749999999999996,4819],[0.4325,4852],[0.5575,4852],[0.7375,5005],[0.3675,5055],[0,5055],[0,5196],[0,5388],[0.0725,5439],[0,5439],[0.5125,5589],[0.345,5627],[0,5627]])
angry25 = np.array([[0,3655],[0,3655],[0,3655],[0.15,3667],[0,3667],[0.06999999999999999,3667],[0,3667],[0,3837],[0,3837],[0,3837],[0.195,5355],[0.13999999999999999,5553],[0,5720],[0.16,5918],[0,6073],[0.5325,6148],[0,6247],[0.027499999999999997,6415]])
angry26 = np.array([[0,28015],[0,28603],[0,28603],[1,28612],[0.7274999999999999,28612],[0.8724999999999999,28612],[0.72,28619],[0.5175,28619],[0.86,28619],[0.1425,28619],[1,28804],[0.8875,28804],[1,28827],[0.2625,28827],[0,28827],[1,28827],[0,29005],[0,29005],[0,29030],[1,29030],[0,29030],[0,29030],[0,29338],[1,29344],[0,29344],[0,29469],[0,29469]])
angry27 = np.array([[0,4180],[0.045,4319],[0.065,4319],[0.1275,4785],[0,5660],[0,5660],[0.355,5671],[0.16999999999999998,5671],[0.02,5889],[0.02,5889],[0,5889],[0,6457],[0.5449999999999999,6478],[0.034999999999999996,6478],[0.02,6489],[0.6275000000000001,6489],[1,6489],[0,7098],[0.0825,7106],[0.39999999999999997,7106],[0.02,7126],[0.16249999999999998,7126],[1,7126],[0,7618],[0.2075,7640],[0.005,7640],[0.32749999999999996,7646],[0.005,7646],[0.08,7646],[0,8147],[1,8160],[0,8160],[0.8925,8160],[0.045,8941],[0.0675,8947],[0.03,8947],[0,9108],[0.32749999999999996,9226],[0,9562],[1,9641],[0.0775,9641],[0,10261],[0.8399999999999999,10274],[0.5825,10274],[0.59,10920],[0.5725,10940],[1,10940],[0.51,10951],[0.975,10951],[0,10951],[0,11362],[0.4774999999999999,11366],[0,11366],[0.36499999999999994,11388],[0,11388],[1,11388],[0,11665],[0,11665],[0.02,11679],[0.605,11679],[0,11679],[0,11948],[0,11948],[0.1925,11974],[0.25749999999999995,11974],[0,11974],[0,12477],[0.3875,12481],[0.6325,12481],[0,12945],[0,12945],[1,13106],[1,13257],[0.21999999999999997,13257],[0.5599999999999999,13296],[0.6225,13296],[1,13380],[0.6275000000000001,13535],[1,13535],[1,13535],[0,14054],[0,14054],[0,14599],[1,14648],[0,14648],[1,14658],[0,14658],[0,14658],[0,14999],[1,15005],[0,15005],[0.94,15005]])
angry28 = np.array([[0,829],[0,1211],[0,1211],[0.6549999999999999,1221],[0,1221],[1,1221],[0,1582],[0.1775,1587],[0,1587],[0,1587],[0.8624999999999999,1947],[0,1947],[0.034999999999999996,1947],[1,1947],[0.13249999999999998,2375],[0.6149999999999999,2383],[1,2383],[1,2383],[0,2798],[0,2798],[0,2798],[1,3351],[0.5,3351],[0.36499999999999994,3351],[1,3351],[0.13,3431],[0.8024999999999999,3431],[1,3431],[0.02,3870],[0.565,3870],[0.0775,3870],[0.18249999999999997,4083],[0.03,4083],[0.2425,4229],[0.0725,4229],[0.475,4405],[0,4405],[0.02,4554],[0,4554],[0.02,4639],[0.2825,4639],[0.19999999999999998,4639],[0,4927],[0,4927],[0,4949],[1,4949],[0,4949],[0,5647],[0.845,5659],[0.10999999999999999,5659],[0.125,6199],[0.02,6199],[0.10499999999999998,6199],[0,6541],[0,7148],[1,7168],[1,7168],[0,7657],[1,7664],[1,7664],[0.02,8217],[0.17500000000000002,8217],[0,8415],[0.3525,8438],[0.2925,8438],[0.46499999999999997,8458],[0.43749999999999994,8458],[0,8458],[0,8772],[1,8792],[0,8792],[1,8792],[0,9236],[0,9236],[1,9254],[1,9254],[0,9254],[1,9579],[0.825,9579],[0.845,9596],[1,9596],[0,9596],[0,9891],[0,9891],[0,9912],[1,9912],[1,9912],[0,10116],[0,10116],[0,10116],[0,10326],[0,10326],[0,10326],[1,10510],[0,10510],[1,10510]])
angry29 = np.array([[0,5640],[0,6274],[0,6744],[0,7274],[0.1775,7457],[0.27499999999999997,8227],[0.02,8239],[0.22499999999999998,8239],[0,8500],[0,8500],[0,8514],[0.21749999999999997,8514],[0.04,8514],[0,8749],[0.23,8760],[0.5874999999999999,8760],[0.1675,8763],[0.5225,8763],[0,8763],[0,8992],[0,8992],[0.20999999999999996,9007],[0,9007],[0.6325,9007],[0,9222],[0,9222],[0.13,9232],[0.25,9232],[0.049999999999999996,9232],[0,9713],[0,9713],[0.08750000000000001,9730],[0.6074999999999999,9730],[0.4825,9730],[0.15,9946],[0.2925,9946],[0.05249999999999999,9959],[0.02,9959],[0,9959],[0,10204],[0,10204],[0,10216],[0,10216],[0.36499999999999994,10216],[0.47,10532],[0.3875,10557],[0.3325,10557],[0.355,10609],[0.165,10609],[0,10881],[0,11130],[0,11138],[0.30749999999999994,11138],[0.015,11349],[0.16249999999999998,11349],[0.2675,11349],[0.115,11576],[0.375,11576],[0,11798],[0,11798],[1,11801],[0.1225,11801],[0.6275000000000001,11801],[0,12040],[0,12040],[0,12417],[0,12421],[0.3625,12421],[0,12717],[0,12717],[0,12966],[0,12966],[0,12966],[0,13322],[0,13322],[0,13611],[1,13623],[1,13623],[0.0825,13860],[1,13860],[0.1525,14059],[0.09499999999999999,14066],[1,14066],[0,14482],[0.36,14760],[0,15119],[0,15119],[0,19672],[0.0075,19692],[0,19692]])
angry30 = np.array([[0,4092],[0.95,4807],[0.9975,4811],[0.3125,4811],[0,5568],[0,5568],[0,5568],[0,6130],[0,6717],[1,6729],[0.6975,6729],[0,7338],[0,7338],[0,8259],[0.38249999999999995,8316],[0.005,8316],[0.0325,8476],[0.7649999999999999,8476],[0.49,8692],[0.385,8759],[0.46499999999999997,8759],[0.0725,8781],[0.1025,8875],[0,8875],[0.02,8958],[0,8958],[0,9804],[0,9804],[0,9804],[0,10645],[0,10645],[0,10645],[0,10663],[0,10663],[0,10663],[0,11089],[0,11089],[0,11097],[1,11097],[0.0775,11097],[0,11296],[0,11296],[0,11296],[0,11492],[0,11503],[0,11503],[0,11503]])
angry31 = np.array([[0,2322],[0,2404],[0.05499999999999999,2449],[0.1175,2631],[0.034999999999999996,2874],[0.0325,3035],[0,3214],[0.024999999999999998,3399],[0.16,3450],[0,3576],[0.0075,3756],[0.27499999999999997,3801],[0.024999999999999998,3801],[0.01,3970],[0.01,3980],[0.18,3980],[0,4118],[0.205,4243],[0,4359],[0.11249999999999999,4433],[0.18999999999999997,4433]])
angry32 = np.array([[0,4811],[0,4904],[0.02,4978],[0.015,5067],[0.0225,5235],[0.21999999999999997,5312],[0.024999999999999998,5406],[0.02,5464],[0.04,5464],[0,5582],[0.135,5648],[0.042499999999999996,5746],[0.1475,5832],[0,5941],[0.0075,6000],[0.235,6000],[0,6126],[0,6171],[0.13999999999999999,6171],[0,6327],[0.08,6372],[0.08,6372],[0,6535],[0.012499999999999999,6590],[0.23,6590],[0,6718],[0.06,6932],[0,7130],[0.18,7294],[0.09,7332],[0.042499999999999996,7332],[0.08,7461],[0.02,7541],[0.41999999999999993,7541]])
angry33 = np.array([[0,1766],[0,1865],[0,1993],[0.049999999999999996,2079],[0.1175,2079],[0,2193],[0.02,2253],[0.0675,2253],[0,2392],[0.012499999999999999,2426],[0.1675,2426],[0,2525],[0.1725,2601],[0,2719],[0,3041],[0.1575,3292],[0.09,3409],[0.1525,3493],[0.1075,3581],[0.2675,3751],[0.047499999999999994,3918],[0,4061],[0.034999999999999996,4068],[0.08750000000000001,4068]])
angry34 = np.array([[0,1664],[0,1762],[0.18999999999999997,1842],[0.15,1999],[0,2397],[0.065,2440],[0,2440],[0.012499999999999999,2599],[0,2665],[0,2742],[0.0225,2841],[0.03,3007],[0.11249999999999999,3085],[0.06999999999999999,3201],[0.02,3263],[0.1025,3263],[0.17500000000000002,3385],[0.02,3442],[0.10499999999999998,3442],[0.0075,3625],[0.13749999999999998,3625],[0,3748],[0.05249999999999999,4005],[0.024999999999999998,4060],[0.0725,4060]])
angry35 = np.array([[0,1751],[0,1908],[0.005,2022],[0.1025,2076],[0.03,2273],[0.155,2440],[0.02,2519],[0.1525,2591],[0,2741],[0,2783],[0.0975,2783],[0,2974],[0.02,3002],[0,3002],[0.02,3226],[0.06,3226],[0.05249999999999999,3241],[0,3241],[0.02,3274],[0,3274],[0.27249999999999996,3357],[0,3469],[0.042499999999999996,3469],[0.2375,3558],[0.335,3752],[0.0975,3774],[0.024999999999999998,3774],[0,3918],[0.005,4041],[0.135,4186]])
angry36 = np.array([[0,2938],[0,3120],[0,3311],[0.34750000000000003,3371],[0,3371],[0.25749999999999995,3780],[0,3884],[0,4114],[0.32249999999999995,4306],[0,4614],[0.11249999999999999,4672],[0.3575,4672],[0.115,5034],[0.4225,5224],[0.41999999999999993,5582],[0.2825,5877],[0,6070],[0.355,6245],[0.26,6378],[0,6665],[0.29,6698],[0.7949999999999999,6698],[0.43749999999999994,7042],[0.1875,7062],[0.1925,7062],[0,7337],[0,7565],[0,7881],[0.03,7947],[0.24,7947],[0,8241],[0.08,8265],[0.1775,8265],[0,8481],[0.8475,8558],[0,8890],[0.33999999999999997,8908],[0.485,8908],[0.135,9234],[0.02,9317],[0.475,9317],[0.335,9498],[0.7325,9597],[0,9597],[0.4325,9892],[0.63,9920],[0.27999999999999997,9920]])
angry37 = np.array([[0,1881],[0.02,1958],[0.36499999999999994,1958],[0,2182],[0.024999999999999998,2250],[0.2425,2250],[0.295,2587],[0.33749999999999997,2587],[0,2946],[0.27249999999999996,3060],[0,3298],[0,3482],[0,3747],[0.2225,3842],[0,4055],[0,4372],[0.4075,4483],[0,4781],[0.62,4901],[0,5130],[0,5441],[0,5538],[0.16999999999999998,5841],[0.22499999999999998,6065],[0,6465],[0.385,6707],[0,6902],[0.3125,6952],[0.40249999999999997,6952],[0.2125,7232],[0.02,7309],[0.49249999999999994,7309],[0,7511],[0.26499999999999996,7612],[0,7799],[0,8134],[0,8290],[0,8560],[0,8785],[0,9036],[0.32249999999999995,9043],[0.5425,9043],[0,9409],[0.1075,9488],[0.1175,9488],[0.6575000000000001,9675],[0.1175,9767]])
angry38 = np.array([[0,1595],[0.205,1648],[0.2675,1648],[0,1978],[0.17500000000000002,1990],[0.32749999999999996,1990],[0,2427],[0.315,2531],[0,2767],[0.24749999999999997,2873],[0,3040],[0.6375,3192],[0.72,3520],[0.17500000000000002,3520],[0,3879],[0.27,3890],[0.2775,3890],[0,4202],[0.1425,4293],[0,4458],[0,4613],[0,4777],[0,4914],[0.205,5208],[0.5575,5208],[0,5420],[0.2075,5424],[0.31,5424],[0,5812],[0,5958],[0,6310],[0.2925,6417],[0.3125,6417],[0,6602],[0.345,6660],[0.34249999999999997,6660],[0.2875,6835],[0,6926],[0,7039],[0.6074999999999999,7346],[0.5925,7352],[0.5425,7352]])
angry39 = np.array([[0,1486],[0.34750000000000003,1575],[0,1733],[0.05249999999999999,1801],[0.37749999999999995,1801],[0,1959],[0.19749999999999998,2006],[0.045,2006],[0,2355],[0.125,2357],[0.36,2357],[0,2591],[0,2716],[0.51,2832],[0,2962],[0.5225,3087],[0.475,3237],[0.32,3387],[0.3025,3522],[0.19999999999999998,3673],[0,3827],[0.44999999999999996,4108],[0.49499999999999994,4133],[0.1575,4133],[0.235,4376],[0.165,4405],[0.385,4405],[0.405,4603],[0,4716],[0,4831],[0.4225,5208],[0.5,5242],[0.4175,5242],[0,5464],[0,5573],[0.2775,5692],[0,6007],[0.2825,6029],[0.1575,6029],[0,6239],[0.20999999999999996,6286],[0.3125,6286],[0,6624],[0.0975,6706],[0.2625,6706],[0,6864],[0,6981],[0,7242]])
angry40 = np.array([[0,2140],[0.15,2208],[0.09499999999999999,2208],[0,2357],[0.0925,2469],[0,2674],[0.17500000000000002,2704],[0.3125,2704],[0.25,2905],[0,3156],[0.39999999999999997,3450],[0.32749999999999996,3470],[0.2925,3470],[0,3706],[0.2225,3728],[0.23,3728],[0,3954],[0,4090],[0.2975,4209],[0.34249999999999997,4338],[0.22749999999999998,4445],[0,4588],[0.32499999999999996,4708],[0,4891],[0.4774999999999999,5120],[0.37999999999999995,5140],[0.39499999999999996,5140],[0,5395],[0,5573],[0.0225,5692],[0,5856],[0.35000000000000003,5955],[0.1525,6257],[0.027499999999999997,6298],[0.1175,6298],[0,6501],[0.24749999999999997,6526],[0.21749999999999997,6526],[0.425,6776],[0.0775,6856],[0.5149999999999999,6856],[0,7014],[0.16249999999999998,7147],[0.33999999999999997,7228],[0,7428],[0.1225,7480],[0.405,7480],[0,7808],[0.1425,7814],[0.02,7814],[0.21749999999999997,8043],[0,8249],[0.16999999999999998,8424],[0,8734],[0.29,8818]])
angry41 = np.array([[0,697],[0.16999999999999998,848],[0,1137],[0.135,1275],[0.18249999999999997,1573],[0.1925,1712],[0.012499999999999999,1989],[0.145,2122],[0.13249999999999998,2408],[0.065,2544]])
angry42 = np.array([[0,907],[0,1083],[0,1397],[0,1795],[0,1996],[0.39249999999999996,2187],[0,2386],[0,2601],[0,2802],[0,3009],[0.32499999999999996,3409]])
angry43 = np.array([[0,1378],[0.1675,1546],[0,1897],[0.32249999999999995,2103],[0,2274],[0.32249999999999995,2486],[0,2805],[0,3066],[0.285,3287],[0,3468],[0.32,3686],[0.24749999999999997,4382]])
angry44 = np.array([[0,1491],[0,1674],[0.3175,1875],[0,2041],[0,2241],[0.3575,2601],[0.405,2792],[0,2971],[0,3172],[0,3390]])
angry45 = np.array([[0,956],[0,1140],[0,1311],[0.205,1486],[0,1675],[0.01,2032],[0.205,2366],[0.165,2513],[0,2865],[0,3046],[0.1575,3279],[0.27249999999999996,3899]])
angry46 = np.array([[0,2884],[0.2975,3076],[0.39749999999999996,3268],[0,3431],[0.19749999999999998,3742],[0.10999999999999999,3866],[0.22499999999999998,4009]])
angry47 = np.array([[0.1775,1726],[0.3175,1950],[0,2182],[0.415,2409],[0,2588]])
angry48 = np.array([[0,5706],[0,5914],[0.0925,6124],[0.315,6267],[0,6450],[0.1725,6642],[0,6803]])
angry49 = np.array([[0,8231],[0,8414],[0.46,8757],[0,8913],[0,9085],[0,9260],[0.045,9856],[0,10066]])
angry50 = np.array([[0,9172],[0.13249999999999998,9349],[0.23,9648],[0,9817],[0.1675,10122],[0,10267],[0.005,10429],[0.2675,10768],[0.7224999999999999,11150],[0,11295],[0,11455],[0,11632],[0,11891]])

fear1 = np.array([[0,4604],[0,5728],[0,7197]])
fear2 = np.array([[0,3535],[0.0575,3887],[0,5779],[0.06999999999999999,6172]])
fear3 = np.array([[0,3840],[0,4842],[0,5249],[0.024999999999999998,5723],[0,6100],[0,6382]])
fear4 = np.array([[0.042499999999999996,3696],[0,5629],[0,5821],[0.05249999999999999,6300],[0,8070]])
fear5 = np.array([[0.065,3122],[0,5178],[0.012499999999999999,7025]],)

fear7 = np.array([[0,2174],[0.012499999999999999,4376],[0.0375,6245],[0,8073]])
fear8 = np.array([[0,3783],[0,5057],[0,6676],[0.165,8046],[0.1525,9545]])
fear9 = np.array([[0,5785],[0,7323],[0,8952],[0,11184]])
fear10 = np.array([[0,6437],[0,8335],[0,10866],[0,11882]])
fear11 = np.array([[0,13158],[0,13407],[0,13620],[0,13883],[0,14115],[0,14359],[0.0975,14654],[0.015,14877],[0,15131],[0.01,15376],[0.08,15661],[0,15909],[0.047499999999999994,16183],[0,16468],[0.08,16928],[0.10499999999999998,17162],[0,17438],[0,17668],[0.0075,18078],[0.03,18194]])
fear12 = np.array([[0,2414],[0.1175,2814],[0,3089],[0,3337],[0.06999999999999999,3593],[0,3896],[0,4157],[0,4539],[0.16249999999999998,4867],[0.1075,5155],[0,5449],[0,5741],[0.125,6067],[0.075,6308],[0,6567],[0.024999999999999998,6579],[0,6579],[0,7002],[0.08,7273],[0,7577],[0,7841]])
fear13 = np.array([[0,3249],[0.04,3570],[0.045,3845],[0.06999999999999999,4058],[0.012499999999999999,4347],[0.0575,4603],[0,4788],[0.0575,5041],[0.01,5301],[0.042499999999999996,5536],[0.0325,5756],[0.017499999999999998,6001],[0.0225,6205],[0.005,6434],[0,6664],[0.034999999999999996,6927],[0.02,7183],[0,7394]])
fear14 = np.array([[0,5393],[0.0725,5650],[0.049999999999999996,5881],[0.0075,6115],[0,6298],[0,6536],[0.02,6736],[0.11249999999999999,6948],[0.075,7152],[0.0575,7365],[0.042499999999999996,7536],[0.0575,7731],[0,7911],[0.042499999999999996,8128],[0,8310],[0,8512],[0,8900]])
fear15 = np.array([[0,10819],[0,11034],[0,11246],[0.115,11468],[0,11650],[0,11838],[0.0975,12074],[0,12277],[0.1425,12471],[0.125,12693],[0.115,12906],[0,13130],[0,13513],[0,13710],[0.115,13941],[0,14130],[0.02,14178],[0.03,14178],[0.165,14378],[0.12,14586],[0.12,14816],[0,14998]])
fear16 = np.array([[0,1445],[0,1985],[0,2433],[0,2945],[0,3363],[0,3938],[0,4372],[0,4825]])
fear17 = np.array([[0,3671],[0,5491],[0,6081],[0.09,6603],[0.16,7091],[0.1025,7564],[0.05249999999999999,8041],[0,8479],[0,9016],[0,9524]])
fear18 = np.array([[0,3827],[0,4034],[0,4250],[0,4451],[0.17500000000000002,4692],[0.049999999999999996,4692],[0.02,4699],[0.02,4699],[0,4699],[0.042499999999999996,4911],[0.02,4929],[0.015,4929],[0.02,4936],[0.0825,4936],[0,4936],[0.0025,5123],[0.0025,5130],[0,5130],[0,5544],[0.034999999999999996,5738],[0.02,5744],[0.08,5744],[0.08750000000000001,5744],[0,5928],[0.015,5928],[0,5934],[0.042499999999999996,5934],[0.049999999999999996,5934]])
fear19 = np.array([[0,5709],[0,5725],[0.16249999999999998,5725],[0,5912],[0,6215],[0.0325,6412],[0.017499999999999998,6840],[0.017499999999999998,6851],[0.195,6851],[0.017499999999999998,6866],[0.0625,6866],[0.13999999999999999,7054],[0.23249999999999998,7058],[0,7058],[0.2925,7068],[0,7068],[0.08499999999999999,7068],[0,7068],[0,7285],[0.2025,7285],[0.017499999999999998,7507],[0.017499999999999998,7519],[0.1275,7519],[0.049999999999999996,7519],[0,7519],[0,7732],[0.2225,7742],[0,7742],[0,7967],[0.024999999999999998,7994],[0,7994],[0,7994],[0.16,7994]])
fear20 = np.array([[0,3787],[0.10499999999999998,3808],[1,3808],[0.10499999999999998,4005],[0.12,4013],[1,4013],[0.024999999999999998,4231],[0.1025,4241],[0.10999999999999999,4241],[0,4424],[0,4611],[0.32,4810],[0,4932],[0.5325,5007],[0.58,5208],[0.09999999999999999,5293],[0,5616],[0,5808],[0.64,5834],[0,5834]])
fear21 = np.array([[0,9801],[0.0225,9813],[0,9813],[0.0225,9827],[0,9827],[0.585,9827],[0.2125,10782],[0,11142],[0,11142],[0.027499999999999997,11282],[0.1675,11282],[0.2225,11282],[0.02,11401],[0.08750000000000001,11401],[0.6124999999999999,11401],[0.505,11546],[0,11546],[0.5775,11566],[0.015,11566],[0.08,11566],[0.02,11697],[0.1525,11697],[0.6375,11697],[1,11851],[0.047499999999999994,11851],[1,11872],[0.315,11872],[0,11872],[0.135,11980],[0.24,11980],[0.385,12060],[0.29,12060],[0.37749999999999995,12502],[0,12502],[0,12875],[0.7875,12904],[0.024999999999999998,12904],[0.485,13062],[0.7375,13062],[0.425,13383],[0.335,13418],[0.0775,13418],[0.0775,13529],[0.565,13529],[0,13659],[0.665,13659],[0.9924999999999999,13692],[0.32249999999999995,13692],[1,14019],[0,14019],[0,14541],[0,14541],[0.13,14605],[0.1675,14605],[0.1225,14605],[0.135,15422],[0,15704],[0.17500000000000002,15713],[0.5299999999999999,15713],[0.02,15830],[1,15830],[0,15830],[0.5874999999999999,16165],[0.8374999999999999,16209],[0,16209],[1,16244],[0.065,16244],[0.042499999999999996,16244],[0.16,16520],[0.195,16586],[0,16586],[0.02,16668],[1,16668],[0.05499999999999999,16884],[0.075,17112],[0,17112]])
fear22 = np.array([[0,532],[0.0725,622],[0.125,622],[0.02,895],[0.27249999999999996,895],[0.3125,895],[0.5025,1613],[0.02,1617],[1,1617],[1,1622],[0.09499999999999999,2356],[0.4975,2356],[0.2675,2356],[0.7025,2677],[0.2025,2677],[0.345,2948],[0.3125,2948],[0,2948],[0.2975,3188],[0.22749999999999998,3188],[0,3188],[0.0725,4139],[0,4139],[0.315,4139],[0.5575,4209],[0.1025,4209],[0.5475,4218],[0.5425,4218],[0.02,4218],[0.5675,4236],[0.29,4236],[0.7474999999999999,4236],[0.11249999999999999,4236],[0.02,4336],[1,4336],[1,4336],[1,4469],[0,4469],[0.02,4559],[0,4559],[0.02,4660],[0.86,4660],[1,4660],[0.02,4793],[0.27249999999999996,4793],[0.02,4812],[0.4975,4812],[1,4812],[0.3125,4842],[0,4842],[0.26499999999999996,4946],[1,4946],[1,5059],[0.39,5059],[1,5088],[0.865,5088],[0,5088],[0.02,5185],[0,5185],[0.82,5212],[1,5212],[0.32,5473],[0.33749999999999997,5555],[0.9349999999999999,5555],[0.39,5675],[0.405,5675],[0.195,5725],[0,5725],[0.8374999999999999,5938],[0.9975,5968],[0.4275,5968]])
fear23 = np.array([[0,1437],[0,1437],[0,1437],[0.0725,3479],[0,3878],[0.03,4327],[0.042499999999999996,4327],[0.03,4341],[0.042499999999999996,4341],[0.0225,4341],[0.0575,4554],[0,4554],[0.02,4739],[0.047499999999999994,4739],[0.0975,4739],[0.02,5056],[0.02,5056],[0.017499999999999998,5056],[0.0375,5231],[0,5231],[0.02,5493],[0.09,5493],[0.0075,5493],[0.0325,5734],[0.01,5734],[0.03,5765],[0.012499999999999999,5765],[0.0225,5765],[0.05499999999999999,5981],[0.09999999999999999,5991],[0,5991],[0.05249999999999999,6056],[0.0025,6056],[0.042499999999999996,6056],[0.0675,6199],[0.0625,6199],[0.0625,6213],[0,6213],[0.0725,6275],[0,6275],[0.2025,6275],[0.215,6392],[0.2525,6405],[0,6405],[0.06999999999999999,6465],[0.21999999999999997,6465],[0.5175,6600],[0,6600],[0.6675,6619],[0,6619],[0.0075,6619],[0.02,6721],[0.02,6721],[0.6,6721],[0.6074999999999999,6848],[0,6848],[0.69,6874],[0,6874],[0.0325,6874],[0.02,6960],[0.02,6960],[0.8175,6960],[0.45249999999999996,7114],[0.01,7114],[0,7114],[0.02,7261],[0.0575,7261],[0.64,7261],[0.49,7441],[0.26499999999999996,7441],[0.39749999999999996,7476],[0.26499999999999996,7476],[0,7476],[0.12,7672],[0.3625,7672],[0.4225,7793],[0.735,7793],[0.047499999999999994,7793],[0.0825,7941],[0.047499999999999994,7941],[0.7925,7941],[0.02,8300],[0.02,8300],[1,8300],[1,8482],[1,8482],[0.2525,8482],[0.02,8698],[0.43749999999999994,8698],[0.02,8800],[1,8800],[0.02,8851],[1,8851],[0.5449999999999999,8851],[1,8981],[0.8875,8981],[0,8981],[0.6525,9079],[1,9079],[0.705,9168],[1,9168],[0.06,9168]])
fear24 = np.array([[0,6571],[0.195,6660],[0.042499999999999996,6660],[0.02,6795],[0.135,6795],[0.75,6795],[0.09499999999999999,12965],[0.08,13097],[0.0975,13097],[0.02,13782],[0.042499999999999996,13782],[0.017499999999999998,13782],[0.027499999999999997,14130],[0.0575,14181],[0,14181],[0.049999999999999996,14475],[0,14475],[0.0325,14511],[0,14511],[0,14511],[0.02,14704],[0.049999999999999996,14704],[0.027499999999999997,14704],[0.03,14941],[0,14941],[0.0225,15337],[0,15337],[0.03,15571],[0,15571],[0.0075,15571],[0.04,15927],[0,15927],[0.04,15950],[0,15950],[0,15950],[0.165,16243],[0.0025,16243],[0.1475,16243],[0.0675,16387],[0.04,16387],[0,16387],[0.215,16594],[0,16594],[0.2625,16594],[0.1875,16751],[0,16751],[0.345,16751],[0.0625,16955],[0,16955],[0.0625,16973],[0,16973],[0.0375,16973],[0.16,17104],[0.38249999999999995,17104],[0.0225,17195],[0.05249999999999999,17195],[0.10999999999999999,17288],[0.38249999999999995,17288],[0.06,17374],[0.8024999999999999,17374],[0,17659],[0.10499999999999998,17870],[0.9524999999999999,17870],[0.725,17984],[0.0625,17984],[0.165,18099],[0.27,18099],[0.02,18203],[0.355,18203],[0.1525,18203],[0,18531],[0.585,18531],[0,18941],[0,18941]])
fear25 = np.array([[0.027499999999999997,2063],[0.027499999999999997,2443],[0.012499999999999999,2443],[0.017499999999999998,2603],[0,2603],[0.017499999999999998,2770],[0.0025,2770],[0,2770],[0.01,2969],[0,2969],[0.01,3264],[0.01,3383],[0,3383],[0.01,3502],[0,3502],[0.01,3577],[0.02,3577],[0,3577],[0.049999999999999996,3865],[0.034999999999999996,3865],[0.049999999999999996,3872],[0.0625,3872],[0,3872],[0.06,3994],[0.1575,4094],[0.15,4114],[0,4114],[0.0375,4210],[0.45499999999999996,4210],[0.8049999999999999,4210],[0,4527],[0,4527],[0.335,4915],[0,4915],[0.5874999999999999,4915],[0.245,5171],[0,5171],[0.09,5334],[0.30749999999999994,5334],[0.02,5473],[0.11249999999999999,5473],[0,5473],[0.02,5632],[0.18249999999999997,5632],[0,5632],[0.22499999999999998,5897],[0.27,5957],[0.305,5957],[0.02,5998],[0.26,5998],[0,5998],[0.335,6156],[0.9424999999999999,6217],[0,6217],[0.03,6318],[0,6318],[0.02,6331],[0,6331],[0.305,6331],[0.02,6447],[0.9424999999999999,6447],[0,6447],[0.02,6768],[0.1925,6768],[0.43,6847],[0.1025,6847],[0.415,6966],[0.4774999999999999,6966],[0.475,7131],[0.045,7131],[0.6124999999999999,7148],[0.31,7148],[0,7148],[0.02,7256],[0.4475,7256],[1,7256],[0,8934],[0.042499999999999996,8940],[0,8940]])
fear26 = np.array([[0,1688],[0,1843],[0.1025,2138],[0.10499999999999998,2297],[0,2588],[0.02,2739],[0,3062],[0.1075,3206],[0,3485],[0,3648],[0,3950],[0.09499999999999999,4104]])
fear27 = np.array([[0,750],[0,901],[0.015,1194],[0,1318],[0.0575,1633],[0.10999999999999999,1792],[0.17500000000000002,2100],[0.08,2251],[0.03,2559],[0.012499999999999999,2703],[0,2975],[0.0625,3150],[0,3451],[0,3583],[0,3851]])
fear28 = np.array([[0,1121],[0.0375,1274],[0,1523],[0.1075,1674],[0.06,1951],[0.0375,2082],[0,2332],[0,2491],[0.049999999999999996,2689],[0,2802],[0.0225,2971],[0,3155],[0,3350],[0,3658],[0.3725,3816],[0,4090]])
fear29 = np.array([[0,495],[0,676],[0.18,963],[0,1126],[0.19999999999999998,1440],[0.0625,1597],[0.1275,1913],[0.0775,2082],[0,2345],[0.042499999999999996,2498],[0,2763]],)
fear30 = np.array([[0,1990],[0,2166],[0.06999999999999999,2499],[0.135,2641],[0,2920],[0.045,3073],[0,3356],[0,3486],[0,3798],[0.034999999999999996,3948],[0.04,4321],[0,4388],[0.049999999999999996,4489],[0.012499999999999999,4638],[0.215,4717]])
fear31 = np.array([[0,6092],[0.0575,6156],[0.1525,6156],[0,6508],[0.13999999999999999,6815],[0.125,7131],[0,7321],[0.0575,7401],[0.125,7401],[0.13,7900],[0.02,8007],[0.27499999999999997,8007],[0,8426],[0.02,8519],[0.01,8519],[0.0675,9123],[0,9416],[0.09,9768],[0,9965],[0.145,10075],[0.03,10619],[0.17500000000000002,10727],[0.08499999999999999,11233],[0.13249999999999998,11345],[0,11852],[0.16249999999999998,12247],[0,12359],[0.15,12984],[0.09499999999999999,13065]])
fear32 = np.array([[0,1820],[0.1475,1889],[0,1889],[0,2244],[0.049999999999999996,2327],[0.01,2327],[0.1725,2873],[0.0075,3270],[0.10999999999999999,3474],[0,3746],[0,3870],[0,4130],[0.15,4221],[0.034999999999999996,4221],[0,4836],[0.065,4862],[0.0725,4862],[0.0675,5267],[0.0975,5366],[0.0025,5684],[0.042499999999999996,5891],[0.0975,6302],[0,6490],[0.017499999999999998,6817],[0.06999999999999999,6890],[0.04,7334],[0.09999999999999999,7404],[0.024999999999999998,7735],[0.012499999999999999,8126],[0.027499999999999997,8518],[0,8766],[0.1475,9269],[0.12,9293],[0.04,9293],[0.024999999999999998,9607],[0.09499999999999999,9764],[0,9992],[0,10257]])
fear33 = np.array([[0,1973],[0.18,2035],[0.115,2035],[0,2586],[0.0225,2590],[0.13999999999999999,2590],[0.03,3137],[0.03,3158],[0.024999999999999998,3158],[0.0375,3517],[0,3679],[0.13,4100],[0.0925,4112],[0.015,4112],[0.0925,4571],[0.10499999999999998,4614],[0,4614],[0.0325,5048],[0,5203],[0.005,5660],[0.0025,6135],[0.02,6559],[0.0975,6654],[0.03,7100],[0.03,7123],[0,7123],[0.05249999999999999,7485],[0.042499999999999996,7626],[0.05499999999999999,8068],[0,8193],[0,8473],[0,8821],[0,9100],[0,9579],[0.02,9677],[0,9677],[0.0225,10163],[0.045,10429],[0,10938],[0.042499999999999996,11227]])
fear34 = np.array([[0.0325,1239],[0.0075,1239],[0,1672],[0,2035],[0,2224],[0.0625,2416],[0,2624],[0,2946],[0.08499999999999999,3092],[0.0075,3092],[0.0325,3488],[0.11249999999999999,4032],[0.04,4166],[0.075,4525],[0.005,4853],[0,5120],[0.017499999999999998,5300],[0.0375,5386],[0.0075,5386],[0.017499999999999998,5723],[0.0325,5822],[0.0775,5822],[0,6206],[0,6485],[0.03,6724],[0.02,6827],[0,6827],[0,7185],[0.045,7191],[0.005,7191],[0.042499999999999996,7574],[0.042499999999999996,7583],[0.005,7583],[0,7944],[0,8283],[0,8516]])
fear35 = np.array([[0,1529],[0.034999999999999996,1537],[0.09,1537],[0,1961],[0.08499999999999999,2141],[0.02,2487],[0,2769],[0,2999],[0.012499999999999999,3411],[0.017499999999999998,3422],[0.03,3422],[0,3812],[0.0025,4078],[0.027499999999999997,4201],[0.0075,4493],[0.045,4786],[0,5086],[0.027499999999999997,5520],[0,5704],[0.0375,5832],[0,6021],[0,6380],[0,6496],[0.06,6929],[0,7089],[0,7345],[0,7814],[0,7970],[0,8357],[0,8860],[0,9104],[0,9582],[0.005,9805],[0,9948],[0,10282],[0,10431],[0,10713],[0,11038],[0,11152],[0,11365]])
fear36 = np.array([[0,1415],[0,2040],[0,2907],[0.04,3190],[0,3974]])
fear37 = np.array([[0,2206],[0.1475,2916],[0,3264],[0,3926],[0,4215],[0,4933],[0.21749999999999997,5625]])
fear38 = np.array([[0,1709],[0,2269],[0,2836],[0.0025,3494],[0,3802],[0,4111],[0.09999999999999999,5116]])
fear39 = np.array([[0,1436],[0.017499999999999998,2152],[0,2728],[0.05249999999999999,3041],[0.042499999999999996,3451],[0.09,3817],[0.024999999999999998,4113]])
fear40 = np.array([[0,3223],[0,3749],[0,4489],[0,4766],[0.09,5040],[0.015,5708],[0,6264],[0.04,6558]])
fear41 = np.array([[0,19341],[0,20543],[0,22001],[0,22891],[0,24524]])
fear42 = np.array([[0,15339],[0,16464],[0,17882],[0,19012],[0,20305]])
fear43 = np.array([[0,13402],[0,15803],[0,17484],[0,19040],[0,20547],[0,22207]])
fear44 = np.array([[0,4366],[0,5701],[0,7596]])
fear45 = np.array([[0,3848],[0,5686],[0,7661],[0,9436]])
fear46 = np.array([[0.0025,3460],[0.01,3866],[0.0725,4070],[0,4644],[0.0025,4869],[0.0575,5462],[0.04,5487],[0,5487],[0.0675,6019],[0,6462],[0,6669],[0.065,7076],[0.024999999999999998,7076],[0.042499999999999996,7829],[0,8897],[0.012499999999999999,9530]])
fear47 = np.array([[0.0325,2545],[0,3110],[0.0375,3895],[0,4313],[0,4332],[0.1725,4332],[0,4820],[0.09499999999999999,5244],[0.1525,5613],[0.09,6061],[0,6475],[0,6861],[0.042499999999999996,7418],[0.049999999999999996,7445],[0,7445],[0.024999999999999998,8019],[0.02,8047],[0,8047],[0.075,8775],[0,9262],[0.08,9660],[0,10007]])
fear48 = np.array([[0,1463],[0,2033],[0.027499999999999997,2433],[0,3059],[0.017499999999999998,3092],[0.017499999999999998,3092],[0.0375,3101],[0.017499999999999998,3101],[0.017499999999999998,3101],[0.02,3718],[0,3718],[0,4611],[0,4960],[0,5461],[0,6225],[0,6468],[0,6743]])
fear49 = np.array([[0.024999999999999998,3087],[0,3911],[0.045,4594],[0,5117],[0,5828],[0.045,5847],[0.0375,5847],[0,6214],[0.042499999999999996,6238],[0.017499999999999998,6238],[0,6912],[0,8070],[0,8070],[0.02,8092],[0.0075,8092],[0.01,8092],[0,8592],[0,8592],[0,8747],[0,8893],[0,9504],[0.02,9663],[0,9988],[0.0075,10151],[0,10151],[0.0075,10157],[0,10157],[0,10157],[0,10244],[0,10244],[0.0075,10244]])
fear50 = np.array([[0,1432],[0,1866],[0.005,2287],[0.005,2296],[0,2296],[0.012499999999999999,3087],[0,4034],[0,4231],[0,4231],[0,4759],[0,4784],[0,4784],[0,5405],[0,5570],[0,5570],[0,6118],[0.015,6148],[0,6148],[0,6875],[0,6989],[0,6989],[0,7517],[0,7767],[0.042499999999999996,7770],[0,7770],[0.047499999999999994,7775],[0,7775],[0,7775],[0,8402],[0,8652],[0.0075,9028],[0.0025,9612],[0,9888],[0,10218],[0,11046],[0,11228],[0,11228],[0.045,11249],[0,11249],[0,11249]])

disgust1 = np.array([[0,15743],[0,16614]])
disgust2 = np.array([[0,2996],[0,3184],[0.0725,3374],[0,4565],[0.06999999999999999,5239],[0,5440]])
disgust3 = np.array([[0,4832],[0,5281],[0,5449],[0.06999999999999999,5790]])
disgust4 = np.array([[0,4185],[0,6268],[0,6453]])
disgust5 = np.array([[0,2459],[0.024999999999999998,2607],[0,3657],[0.08750000000000001,3815]])

disgust11 = np.array([[0,28007],[0,28099],[0,28392],[0,28785],[0,29201],[0,29468],[0,29817],[0,30084]])
disgust12 = np.array([[0,3585],[0.06,4369],[0,4645],[0,5024],[0,5396],[0.017499999999999998,5640],[0,6175]])
disgust13 = np.array([[0,2926],[0,3250],[0.0025,3633],[0,3811],[0,4120],[0.01,4376],[0,4683],[0,4968],[0,5194],[0,5511],[0.0325,5746],[0,6009],[0.06,6273]])
disgust14 = np.array([[0,4100],[0.335,4305],[0.0625,4749],[0,5021],[0,5380],[0,5772],[0.047499999999999994,6063],[0,6437],[0,6760],[0,7026],[0,7342],[0,7618],[0,7939],[0,8235],[0.01,8467]])
disgust15 = np.array([[0,2933],[0,3375],[0,3702],[0.09999999999999999,4121],[0.20999999999999996,4482],[0,4839],[0,5074],[0,5319],[0,5777],[0,6233],[0.015,6580],[0,6901],[0,7212],[0,7542],[0,7831],[0,8095],[0,8719]])
disgust16 = np.array([[0,10654],[0.0075,10844],[0.2225,10849],[0,10849],[0.605,12648],[0,12808],[0.7025,13043],[0.41,13252],[0.2875,13460],[0.40249999999999997,13670],[0.3525,13877],[0.43,14065]])
disgust17 = np.array([[0,4100],[0,4507],[0,4826],[0,5006],[0,5268],[0.16999999999999998,5441],[0.2875,5534],[0,5652],[0,5759],[0,5855],[0.1725,5980],[0.06999999999999999,6060],[0.2125,6185],[0,6392],[0,6461],[0,6586],[0,6770],[0,6858],[0,6951],[0.0725,7045],[0.23,7131],[0,7231],[0.485,7318],[0,7421],[0.81,7499],[0,7590]])
disgust18 = np.array([[0,1925],[0.02,1952],[0.6575000000000001,1952],[0.047499999999999994,2384],[0.047499999999999994,2388],[0.26499999999999996,2388],[0,2582],[0.16249999999999998,2600],[0,2600],[0.02,2612],[0.0075,2612],[0,2612],[0.02,2619],[0.23249999999999998,2619],[0.06999999999999999,2784],[0.05249999999999999,2784],[0.39,2806],[0,2806],[0,3016],[0.23249999999999998,3016],[0.285,3189],[0.05249999999999999,3189],[1,3211],[0,3211],[0.02,3404],[0.0825,3404],[0.5025,3417],[0.02,3417],[0,3417],[1,3417],[0,3579],[0.015,3590],[0,3590],[0.042499999999999996,3609],[0,3609],[0.37,3609],[0.5475,3609],[0.24749999999999997,3807],[0.335,3807],[0.1425,3828],[0.51,3828],[0.005,3828],[0.13749999999999998,3828],[0.1475,4000],[0.1025,4000],[0.02,4019],[0.355,4019],[1,4019],[0.26499999999999996,4019]])
disgust19 = np.array([[0,4860],[0,5147],[0,5417],[0.6975,5679],[0.1675,5850],[0,6129],[0,6312],[0.235,6566],[0,6753],[0.27,7002],[0.0725,7184],[0,7496],[0,7496],[0.2025,7683],[0,7683],[0.0375,7878],[0,7878],[0.10999999999999999,8029],[0.02,8034],[0.03,8034],[0.45249999999999996,8078],[0,8078],[0.0925,8224],[0.02,8249],[0.02,8249],[0,8688],[0,8912]])
disgust20 = np.array([[0.1775,4145],[0.305,4401],[0.7125,4632],[0.6924999999999999,4868],[1,5115],[0.0775,5370],[1,5573],[1,5828],[0.47,6060],[0.075,6267],[0.605,6507],[0.05499999999999999,6730],[0.1575,6968],[0.27,7189]])
disgust21 = np.array([[0,8992],[0.0975,9032],[0.0825,9032],[0.0725,9925],[0.0725,9929],[0,9929],[0.02,10344],[0.02,10344],[1,10344],[0,10866],[0,10872],[0.155,10872],[0,11464],[0.0025,11528],[0.12,11528],[0.0025,11706],[0.2425,11706],[0,11706],[0.0025,12040],[0.02,12040],[0.3025,12040],[0.0025,12214],[0.115,12214],[0,12214],[0.13749999999999998,12660],[0.02,12664],[0.0375,12664],[0,13757],[0.33,13777],[0.0725,13777],[0,14380],[0,14380],[0.0025,14824],[0.115,14985],[0,14985],[0.027499999999999997,14985],[0,15201],[0.02,15201],[0.3025,15201],[0,15744],[0.4425,15752],[0,15752],[0,15752],[0.02,16139],[0,16139],[0,16139],[0,16413],[0,16423],[0.305,16423],[0.32499999999999996,16423],[0,16885],[0.1875,16916],[0.0975,16916]])
disgust22 = np.array([[0,1476],[0.02,1522],[0.19749999999999998,1522],[0.02,2017],[0.02,2017],[0.8749999999999999,2017],[0,2314],[0.245,2323],[0,2323],[0,2631],[0,2631],[0,2870],[0.0925,2876],[0.01,2876],[0.02,3152],[0.01,3152],[0.95,3152],[0.1925,3530],[0.1475,3551],[0.1225,3551],[0.13749999999999998,3565],[0.045,3565],[0.305,3565],[0.02,3891],[0.05249999999999999,3891],[0.22499999999999998,3891],[0.2375,4053],[0.08750000000000001,4053],[0.3,4079],[0.295,4079],[0.012499999999999999,4079],[0.9874999999999999,4372],[0,4372],[0,4621],[0,4621],[0.005,4647],[0.25749999999999995,4647],[0,4647],[0.005,4711],[0.02,4711],[1,4711],[0.2375,5058],[0,5058],[0.02,5163],[0.08,5163],[0.034999999999999996,5163]])
disgust23 = np.array([[0,2851],[0.04,2885],[0.19999999999999998,2885],[0.02,3065],[0.0375,3065],[0.0075,3065],[0.0825,4020],[0.0775,4133],[0.017499999999999998,4133],[0,4718],[0.1275,5368],[0.1925,5553],[0.09999999999999999,5553],[0.02,5750],[0.1275,5750],[0.58,5750],[0.23,6179],[0.1775,6182],[0.02,6183],[0.3575,6186],[0,6186],[0.0375,6226],[0.6174999999999999,6229],[0,6229],[0.042499999999999996,6242],[0.5874999999999999,6244],[0,6244],[0.065,6310],[0.49,6315],[0,6315],[0.0775,6326],[0.34750000000000003,6327],[0,6327],[0.0625,6341],[0.2825,6343],[0,6343],[0.37999999999999995,6499],[0.027499999999999997,6499],[0.02,6520],[0.02,6520],[1,6520],[0,6973],[0.2675,7090],[0,7090]])
disgust24 = np.array([[0,1362],[0.1075,1406],[0.005,1406],[0.02,1586],[0.017499999999999998,1586],[0.0325,1586],[0.0375,2238],[0.205,2238],[0.034999999999999996,2238],[0.02,2848],[0.042499999999999996,2848],[0.1175,2848],[0.0825,3220],[0.012499999999999999,3220],[0.049999999999999996,3287],[0.045,3287],[0.015,3287],[0.02,3564],[0.015,3564],[0.06999999999999999,3564],[0.1275,4230],[0.26499999999999996,4230],[0.145,4239],[0.19999999999999998,4239],[0.05499999999999999,4239],[0.0325,4697],[0.0075,4697],[0,4953],[0.04,5097],[0.0725,5097],[0.09,5220],[0.042499999999999996,5220],[0.017499999999999998,5220],[0.047499999999999994,5398],[0.04,5398],[0.02,5566],[0.0325,5566],[0.0675,5566],[0.32249999999999995,6398],[0.27249999999999996,6398],[0.10999999999999999,6398],[0.1725,6917],[0.02,6917],[1,6917],[0.042499999999999996,7249],[0.8274999999999999,7249],[1,7725],[0,7725],[1,7732],[0,7732],[0.32,7732],[0.21749999999999997,8165],[0.02,8165],[1,8165],[0.02,8434],[1,8434],[0.33749999999999997,8434]])
disgust25 = np.array([[0,1217],[0.10499999999999998,1349],[0.005,1349],[0.034999999999999996,1443],[0.017499999999999998,1443],[0,1443],[0.05249999999999999,2099],[0,2099],[0.02,2119],[0.0025,2119],[0,2119],[0.005,2442],[0.17500000000000002,2442],[0.02,2709],[0.0575,2709],[0.02,2804],[0.05249999999999999,2804],[0,2804],[0.02,3145],[0,3145],[0.09999999999999999,3145],[0.049999999999999996,3294],[0,3294],[0.17500000000000002,3305],[0.11249999999999999,3305],[0.02,3408],[0.02,3408],[0.0375,3408],[0.0825,4239],[0,4239],[0.02,4257],[0,4257],[0,4257],[0,4619],[0,4619],[1,4619],[0,4804],[0.09,4836],[0.1425,4836],[0.02,4969],[0.4425,4969],[0.065,5123],[0,5123],[0.08,5224],[0.05249999999999999,5224],[0.02,5350],[1,5350],[0.2775,5805],[0.3175,5805],[0.3875,5810],[0.2025,5810],[0,5810],[0,6374],[0.24,6399],[0,6399],[0.02,6605],[1,6605],[0,6605],[1,6751],[0,6751],[0.13,6785],[0,6785],[0.02,6869],[0.02,6869],[1,6869],[0.02,6952],[1,6952],[0,6952]])
disgust26 = np.array([[0,4781],[0.012499999999999999,4938],[0.17500000000000002,5022],[0,5255],[0.1275,5653],[0.042499999999999996,5837],[0.02,5855],[0.16,5855],[0.08,5985],[0,6132],[0.23,6223],[0,6333],[0.04,6387],[0.115,6387],[0.0375,6562],[0.09499999999999999,6747],[0.034999999999999996,6878],[0.0775,6969],[0.05499999999999999,7092],[0.1275,7182],[0.06,7292],[0.02,7364],[0.1575,7364],[0,7530],[0.09,7693],[0,7693],[0,7996],[0,8197],[0.25749999999999995,8423],[0.075,8612],[0,8760],[0.0325,8960],[0.13249999999999998,9139],[0.049999999999999996,9314]])
disgust27 = np.array([[0,600],[0,776],[0,1092],[0.0075,1098],[0.16999999999999998,1098],[0,1271],[0,1271],[0,1424],[0,1434],[0.3025,1434],[0.4975,1445],[0.285,1445],[0.1775,1445],[0.015,1595],[0.23,1609],[0.17500000000000002,1609],[0.08,1609],[0,1759],[0.13,1786],[0,1786],[0.215,1786],[0.05249999999999999,1920],[0.05249999999999999,1930],[0.235,1930],[0.02,1941],[0.02,1941],[0.10499999999999998,1941],[0,2088],[0,2088],[0.047499999999999994,2108],[0,2108],[0,2108],[0,2251],[0.0725,2266],[0,2266],[0.0725,2284],[0,2284],[0,2284],[0,2414],[0,2436],[0.16,2436],[0,2436],[0,2583],[0,2600],[0.33999999999999997,2600],[0,2600],[0,2745],[0,2745],[0,2745]])
disgust28 = np.array([[0,2462],[0.0775,2620],[0,2947],[0,2959],[0.34249999999999997,2959],[0.19749999999999998,2959],[0,3101],[0,3112],[0.10999999999999999,3112],[0,3116],[0.02,3116],[0.17500000000000002,3116],[0.16,3264],[0.06999999999999999,3282],[0.045,3282],[0.0675,3282],[0.31,3433],[0.02,3449],[0,3449],[0,3449],[0.3575,3569],[0.205,3597],[0.1075,3597],[0,3717],[0.1425,3765],[0,3765],[0.17500000000000002,3907],[0,4060],[0.24749999999999997,4078],[0.125,4078],[0.115,4092],[0.12,4092],[0,4092],[0.0775,4255],[0,4255],[0.02,4278],[0,4278],[0.09999999999999999,4278],[0.22749999999999998,4418],[0,4418],[0.0825,4434],[0,4434],[0.0075,4434],[0,4595],[0,4595],[0,4609],[0.03,4609],[0.042499999999999996,4609],[0,4758],[0.125,4770],[0,4770],[0.16999999999999998,4789],[0,4789],[0,4789]])
disgust29 = np.array([[0,563],[0.024999999999999998,576],[0,576],[0.065,705],[0.065,722],[0.1425,722],[0.027499999999999997,730],[0.02,730],[0.135,730],[0,1007],[0,1032],[0,1032],[0,1040],[0,1040],[0.125,1040],[0.065,1171],[0.0725,1171],[0.13749999999999998,1171],[0.017499999999999998,1329],[0.017499999999999998,1351],[0.26,1351],[0,1351],[0.045,1506],[0.23,1506],[0.0975,1506],[0.065,1660],[0.155,1681],[0.16999999999999998,1681],[0.05249999999999999,1681],[0,1838],[0.042499999999999996,1844],[0.049999999999999996,1844],[0.02,1852],[0.02,1852],[0.065,1852],[0,2020],[0,2020],[0,2039],[0,2039],[0.08,2039],[0.11249999999999999,2186],[0.02,2191],[0.09999999999999999,2191],[0.024999999999999998,2196],[0.05249999999999999,2196],[0,2196],[0,2372],[0.1875,2372],[0.18999999999999997,2372],[0.012499999999999999,2577],[0.18,2577],[0.012499999999999999,2595],[0.18,2595],[0,2595],[0.22499999999999998,2729],[0.09499999999999999,2734],[0.27499999999999997,2734],[0.02,2734]])
disgust30 = np.array([[0,520],[0.047499999999999994,529],[0,529],[0,529],[0,695],[0,695],[0,695],[0,850],[0,850],[0,1014],[0.2225,1021],[0.26,1021],[0.04,1021],[0,1157],[0.13749999999999998,1180],[0,1180],[0,1274],[0.042499999999999996,1368],[0.305,1368],[0,1537],[0.024999999999999998,1537],[0,1553],[0.024999999999999998,1553],[0.37749999999999995,1553],[0.155,1714],[0.1575,1714],[0.1925,1714],[0,1888],[0.16249999999999998,1888],[0.4774999999999999,1888],[0,2068],[0.08,2087],[0,2087],[0.18249999999999997,2236],[0.0375,2245],[0.045,2245],[0.02,2251],[0.02,2251],[0.315,2251],[0.0975,2415],[0,2415],[0.2225,2415],[0.305,2591],[0,2591],[0.24,2591],[0,2759],[0,2770],[0,2770],[0.3025,2770],[0.39249999999999996,2934],[0.0925,2934],[0.16,2934]])
disgust31 = np.array([[0.08,5628],[0.1575,5656],[0.05499999999999999,5656],[0.0725,6028],[0.04,6438],[0.0625,6466],[0.06999999999999999,6466],[0.02,7077],[0,7338],[0.1275,7920],[0.23249999999999998,7996],[0.08,8569],[0.0775,8621],[0.145,8621],[0.11249999999999999,8903],[0,9162],[0,9653],[0.17500000000000002,9689],[0.017499999999999998,9689],[0.0725,10114],[0.024999999999999998,10173],[0.0075,10173],[0.135,10454],[0.1075,10461],[0.0075,10461],[0.2225,10996],[0.04,11094],[0.08750000000000001,11579],[0.027499999999999997,11620],[0.26,11620],[0.1525,11945]])
disgust32 = np.array([[0,1533],[0.16999999999999998,1538],[0.08,1538],[0.15,2179],[0.0675,2226],[0.08750000000000001,2226],[0,2414],[0.06999999999999999,2600],[0.16249999999999998,2934],[0.075,3062],[0,3785],[0.027499999999999997,3853],[0,3853],[0.042499999999999996,4395],[0.04,4478],[0.034999999999999996,4478],[0.09499999999999999,4654],[0,4730],[0,4979],[0.04,5089],[0.0725,5356],[0.0725,5375],[0.024999999999999998,5375],[0.13749999999999998,5554],[0,5667],[0.0675,5947],[0.215,6119],[0.145,6572],[0,6753],[0,7105],[0.20999999999999996,7419],[0.22499999999999998,7437],[0.0925,7437],[0.1025,7753],[0.1175,7778],[0.18,7778],[0,8123],[0,8304]])
disgust33 = np.array([[0,6371],[0.034999999999999996,6466],[0.0825,6466],[0.16249999999999998,6798],[0.09,6835],[0.08750000000000001,6835],[0.015,7546],[0.0225,7560],[0.0375,7560],[0,8098],[0.02,8205],[0.09499999999999999,8205],[0.047499999999999994,8849],[0.06999999999999999,8926],[0,8926],[0,9115],[0,9273],[0.1525,9379],[0,9598],[0.0225,10015],[0.1175,10060],[0.0625,10060],[0.08,10389],[0.16,10480],[0.165,10613],[0,10749],[0,10965],[0.1075,11222],[0.1075,11234],[0.155,11234],[0,11439],[0.09,11449],[0.12,11449],[0.1425,11874],[0,12074],[0.15,12709],[0,12900],[0.10499999999999998,12905],[0.08750000000000001,12905],[0.1475,13165]])
disgust34 = np.array([[0,2189],[0.047499999999999994,2271],[0.08,2271],[0,2670],[0.1175,2693],[0.03,2693],[0.0075,3171],[0.0625,3196],[0.034999999999999996,3196],[0,3629],[0.13249999999999998,3757],[0,3979],[0.017499999999999998,4088],[0,4288],[0,4672],[0.08,4688],[0.18,4688],[0.034999999999999996,5206],[0.16,5422],[0,5898],[0.02,6015],[0.0775,6015],[0.06999999999999999,6665],[0.0575,6724],[0.1875,6724],[0.012499999999999999,7290],[0,8125],[0.0325,8168],[0.017499999999999998,8168],[0,8397],[0.08750000000000001,8446],[0,8446],[0.1275,8663],[0.0775,8855],[0.1175,9145],[0.18249999999999997,9348],[0.1575,9698],[0,10073],[0.05499999999999999,10140],[0.024999999999999998,10140],[0.0725,10308],[0.1175,10491],[0.047499999999999994,10925],[0.042499999999999996,11008]])
disgust35 = np.array([[0,3042],[0,3465],[0.05249999999999999,3485],[0.047499999999999994,3485],[0,4044],[0.012499999999999999,4302],[0,4861],[0,5047],[0.0375,5301],[0.16249999999999998,5697],[0.0375,5807],[0.1525,5807],[0,6108],[0,6461],[0,6865],[0.08,7253],[0.135,7502],[0,8210],[0.12,8437],[0.0725,9023],[0.017499999999999998,9112],[0,9836],[0,10429],[0,10594],[0,10737],[0.015,10927],[0.09499999999999999,11153],[0.115,11337],[0.0775,11923],[0,12121],[0.18999999999999997,12861],[0.04,12861]])
disgust36 = np.array([[0,2754],[0,3243],[0,3726],[0.0025,4251],[0,4645],[0.042499999999999996,4944],[0.0025,5189]])
disgust37 = np.array([[0.1175,1147],[0,1658],[0,1905],[0.115,2306],[0.1225,2825],[0.0725,3050],[0.1175,3269],[0,3765]])
disgust38 = np.array([[0,941],[0.0575,1329],[0.04,1983],[0.065,2327],[0,2658]])
disgust39 = np.array([[0,2776],[0,3293],[0,4093],[0.13999999999999999,4277],[0.0625,4493],[0.0075,4952]])
disgust40 = np.array([[0,5154],[0.125,5585],[0,6160],[0,6335],[0.027499999999999997,7071],[0,7464],[0.02,7929]])
disgust41 = np.array([[0,5193],[0,6674],[0.06999999999999999,7328],[0,7808],[0,8570]])
disgust42 = np.array([[0.04,2142],[0.02,2898],[0,3905],[0,4081],[0,4507],[0.0075,4712]])
disgust43 = np.array([[0.005,4113],[0.0225,4519],[0.02,4898],[0.02,5537],[0.02,5828],[0,6808],[0,7008],[0.0075,7739],[0.017499999999999998,7893]])
disgust44 = np.array([[0.06999999999999999,3450],[0,4409],[0.005,5197],[0.0575,6049],[0,6657]])
disgust45 = np.array([[0,2041],[0.13749999999999998,3035],[0,4261],[0,5067],[0,6156],[0,7586]])
disgust46 = np.array([[0.0025,2170],[0,3633],[0,4424],[0,5733],[0,5923],[0,6727],[0,7809],[0,7971],[0,8985]])
disgust47 = np.array([[0,3235],[0,3407],[0,4177],[0,5107],[0,5501],[0,5841],[0,6904],[0,7939],[0,8854],[0,9661],[0,10182],[0,11256],[0.09499999999999999,11733]])
disgust48 = np.array([[0,3374],[0,3808],[0,4022],[0,5341],[0.005,5793],[0,6733],[0.0675,7275],[0,8252],[0.075,9351],[0,9593]])
disgust49 = np.array([[0,1278],[0,1692],[0.18249999999999997,2462],[0.005,2942],[0,4069],[0,4596],[0.049999999999999996,5042],[0.042499999999999996,5848],[0.0225,6715],[0,6988],[0,7671],[0,8516]])
disgust50 = np.array([[0,2406],[0.02,2687],[0.024999999999999998,2850],[0,2850],[0,3644],[0.0075,3839],[0,4576],[0.0075,4774],[0,5616],[0.005,6327],[0.01,6512],[0,7352],[0,7969],[0,8846],[0,9219],[0,9704],[0,9801],[0,10705],[0,11053],[0,11929],[0,12637],[0,12829]])

surprise1 = np.array([[0,15357],[0,17050],[0.21749999999999997,17246]])
surprise2 = np.array([[0,4395],[0,4646],[0.0325,4938],[0,5217]])
surprise3 = np.array([[0,6971],[0.1025,10285],[0,10475]])
surprise4 = np.array([[0,4122],[0.4225,4273],[0,5944],[0,6094]])
surprise5 = np.array([[0.15,6437],[0,7050],[0.255,7205]])
surprise6 = np.array([[0,11450],[0,11450],[1,11464],[0,11464],[0,11464],[0,13765],[0.042499999999999996,13788],[0.05249999999999999,13788],[0,14435],[0.125,14435],[0,15213],[0,15213]])
surprise7 = np.array([[0,3116],[0,3121],[1,3121],[0.7925,3121],[0,3146],[0.33999999999999997,3146],[0.027499999999999997,3146],[0,3146],[1,4319],[0,4319],[1,4335],[0,4335],[1,4335],[0,5045],[0.43749999999999994,5045],[0.22749999999999998,5573]])
surprise8 = np.array([[0,3194],[0.0675,3269],[0.4075,3269],[0.0975,4189],[0.027499999999999997,4241],[0.12,4241],[0.02,4309],[0.23249999999999998,4309],[0.06,4309],[0.02,4893],[0.1025,4974],[0.2225,4974],[0.02,4989],[0.1925,4989],[0,4989],[0.0375,5680],[0.027499999999999997,5706],[0.0375,5706],[0.02,5730],[0.0325,5730],[0.04,5730]])
surprise9 = np.array([[0,2995],[0.065,3076],[0.11249999999999999,3076],[0.16999999999999998,3118],[0.29,3118],[0,3847],[0.047499999999999994,3867],[0.0725,3867],[0.0925,3879],[0.02,3879],[0.0975,3879],[0.0375,3902],[0.02,3902],[0.02,3902],[0.34750000000000003,3902],[0,4852],[0,4870],[0,4870],[0,4870],[0.045,5502],[1,5502],[0.10499999999999998,5502],[0.02,5507],[1,5507],[0.54,5507],[0,5507]])
surprise10 = np.array([[0,5454],[0,5454],[0,5454],[0,5466],[0,5466],[0.034999999999999996,5466],[0,5466],[0.5925,5466]])
surprise11 = np.array([[0,3722],[0.02,3747],[0.1725,3747],[0,4190],[0.0575,5115],[0,6456]])
surprise12 = np.array([[0,7032],[0,8725],[0,10425],[0.0225,10534]])
surprise13 = np.array([[0,2742],[0,2840],[0.0075,2880],[0.047499999999999994,2880],[0,3508],[0,4276],[0,5495],[0.02,6194],[0,6792]])
surprise14 = np.array([[0,3071],[0.02,3095],[0.13249999999999998,3095],[0.245,3339],[0.16999999999999998,3406],[0,3914],[0,4272],[0.05499999999999999,4947],[0,5806],[0,6132]])
surprise15 = np.array([[0,2946],[0,3116],[0.0025,3181],[0,3646],[0.05499999999999999,3878],[0,4036],[0.1475,4605],[0.0375,5155],[0.0575,5787],[0,6388]])
surprise16 = np.array([[0,28101],[0.36499999999999994,28121],[1,28121]])
surprise17 = np.array([[0,2206],[0,2206],[0,2216],[0,2216],[0,2216],[0,2220],[1,2220],[0.26,2220],[0.03,2220],[0.1075,2220],[0,2527],[1,2527],[1,2527],[0,2541],[1,2541],[0.745,2541],[0,2541]])
surprise18 = np.array([[0,2925],[0.05499999999999999,2936],[0.015,2936],[0,3065],[0,3089],[0,3089],[0.005,3089],[0.024999999999999998,3089],[0.015,3095],[0.0325,3095],[0.02,3095],[0,3095],[0.0025,3179],[0.0675,3191],[0,3191],[0.02,3210],[0,3210],[0,3265],[0,3339],[0.034999999999999996,3339],[0.02,3352],[0.10999999999999999,3352],[0.015,3352],[0,3373],[0.024999999999999998,3463],[0.0925,3478],[0,3478],[0.5125,3490],[0,3490],[0.02,3490],[0.27999999999999997,3502],[0,3502],[0,3502],[0,3599],[0.08750000000000001,3608],[0,3608],[0,3608],[0.065,3642],[0,3642]])
surprise19 = np.array([[0,3541],[0.26499999999999996,3568],[0.19999999999999998,3568],[0.0725,3966],[0.13749999999999998,3966],[0.047499999999999994,4657],[0.08499999999999999,4657],[0.02,4677],[0.375,4677],[0.08499999999999999,4677],[0,4677],[0.08499999999999999,4677]])
surprise20 = np.array([[0,3035],[0.0625,3059],[0,3059],[0.22749999999999998,3182],[0,3337],[0,3337],[0.46,3493],[0.32499999999999996,3500],[0.0825,3500],[0.32,3651],[0.27249999999999996,3662],[0.1425,3662],[0.27249999999999996,3946],[0.145,3971],[0.049999999999999996,3971],[0.024999999999999998,4093],[0.027499999999999997,4261],[0.075,4406],[0.08750000000000001,4433],[0.0975,4433],[0.34249999999999997,4533]])
surprise21 = np.array([[0,15566],[0,15566],[0.045,15950],[0.32,16035],[0.7299999999999999,16035],[0,16398],[0.26,16412],[0,16412],[0.02,16416],[0.2025,16416],[0,16416],[0,16712],[0.09,16718],[0.1925,16718],[0,16832],[0,16832],[1,16846],[0.3025,16846],[0,16846],[0.01,17008],[0.3525,17181],[0.02,17194],[0.165,17194],[0.02,17211],[0.02,17211],[0,17211],[0,18190],[0.027499999999999997,18230],[0.1025,18230],[0.02,18246],[0.19749999999999998,18246],[0.24749999999999997,18246],[0.0675,18320],[1,18370],[0.075,18370],[1,18381],[0.33,18381],[0.165,18381],[1,18413],[1,18413],[0.18999999999999997,18413],[0.445,18563],[0.61,18580],[0,18580],[0.66,18589],[0,18589],[0.02,18589],[0.66,18600],[0.9225,18600],[0.7849999999999999,18600]])
surprise22 = np.array([[0,790],[0.0725,815],[0.0325,815],[0,1779],[0,2557],[0,3424],[0.02,3430],[0.065,3430],[0.7274999999999999,4008],[0,4136],[0,4146],[0.3875,4146],[0.5025,4483],[0.02,4493],[1,4493],[0,4723],[0,5063],[0.02,5084],[1,5084],[0.11249999999999999,5305],[0.5825,5332],[0,5332],[0,5480],[0,5480],[0,5867]])
surprise23 = np.array([[0.41999999999999993,2694],[0.02,2699],[0.2025,2699],[0.04,3400],[0.11249999999999999,3400],[0.1875,4095],[0.0375,4166],[0,4166],[0.0925,4553],[0.03,4787],[0,5011],[0.315,5486],[0.315,5500],[0.335,5500]])
surprise24 = np.array([[0,662],[0.145,690],[0.042499999999999996,690],[0,1229],[0,1229],[0,1232],[0,1232],[0.7775,1232],[0,1441],[0,1441],[0,1441],[0,1652],[0.06999999999999999,1664],[0.065,1664],[0.1075,1667],[0.02,1667],[0,1667],[0,2602],[0,2619],[0,2619]])
surprise25 = np.array([[0,1469],[0.13,2030],[0.4075,2047],[0,2047],[0,2703],[0,2717],[0.8724999999999999,2717],[0,2844],[0,2844],[0,2975],[0,2975],[0,3395],[0,4247]])
surprise26 = np.array([[0,1355],[0.165,1516],[0.09499999999999999,1831],[0,1980],[0,2151],[0.2075,2156],[0.24749999999999997,2156],[0.08750000000000001,2315],[0.02,2338],[0.32,2338],[0,2472],[0,2668],[0,2685],[0.33999999999999997,2685],[0,2849],[0,2862],[0.25,2862],[0.1725,3028],[0.18999999999999997,3192],[0.02,3211],[0.16,3211],[0,3389],[0.03,3402],[0.3025,3402],[0,3573],[0.27999999999999997,3588],[0,3761],[0.165,3784],[0.10999999999999999,3784],[0,3982],[0,4186],[0,4191],[0.37749999999999995,4191],[0,4396],[0,4396]])
surprise27 = np.array([[0,737],[0,949],[0.25,1146],[0,1338],[0,1485],[0,1796],[0,1968],[0,2127],[0,2264],[0,2666]])
surprise28 = np.array([[0,1350],[0.0925,1525],[0,1798],[0,1951],[0,2114],[0,2289],[0,2590],[0.19749999999999998,2739],[0.065,3069],[0,3240],[0,3379],[0.23249999999999998,3541],[0,3851]])
surprise29 = np.array([[0,2609],[0,2941],[0,3121],[0,3298],[0,3511],[0,3660],[0,3843],[0,4011],[0,4198],[0,4419]])
surprise30 = np.array([[0.0225,948],[0.042499999999999996,1122],[0,1282],[0,1447],[0.08499999999999999,1631],[0,1799],[0,2131],[0,2308],[0,2651],[0,2815],[0.0725,3023],[0,3228]])
surprise31 = np.array([[0.1225,4061],[0,4168],[0,4324],[0,4583],[0.335,4681],[0.23249999999999998,4802],[0,4924],[0.24749999999999997,5026],[0.23,5156],[0.27999999999999997,5265],[0,5415],[0,5543],[0.26499999999999996,5675],[0,5793],[0,5909],[0.125,5995],[0.2125,6186],[0.115,6200],[0.215,6200],[0,6445],[0.02,6572],[0.047499999999999994,6681],[0.13249999999999998,6794],[0.0375,6884],[0.02,6884],[0,7012],[0.065,7099],[0.2625,7099],[0,7266],[0.02,7352],[0.2875,7352],[0,7667],[0.1025,7693],[0.1425,7693],[0,7894],[0.33749999999999997,7996],[0.26,8124],[0.205,8250],[0.0575,8293],[0.0025,8293],[0,8472],[0.1875,8500],[0.2025,8500],[0.2875,8737],[0.2375,8764],[0.017499999999999998,8764],[0.23249999999999998,8953],[0,9054],[0.3,9153]])
surprise32 = np.array([[0,1736],[0.19749999999999998,1770],[0.23249999999999998,1770],[0.2525,1959],[0.3325,2080],[0,2203],[0.1425,2338],[0,2435],[0.1925,2786],[0.005,2912],[0.0325,3022],[0,3170],[0.0625,3236],[0,3236],[0.1525,3560],[0,3744],[0,3964],[0.06,4029],[0.125,4029],[0,4180],[0.02,4240],[0.0975,4240],[0.0575,4387],[0.05249999999999999,4430],[0.0625,4430],[0.04,4763],[0.049999999999999996,4831],[0.0025,4831],[0.08499999999999999,5031],[0,5189],[0.02,5265],[0.09999999999999999,5265],[0.0325,5496],[0.0975,5761],[0,5932],[0.0575,5970],[0.08,5970],[0.034999999999999996,6168],[0,6538],[0.0975,6675],[0,6798],[0.0775,6996],[0,7248],[0.06,7278],[0.0725,7278]])
surprise33 = np.array([[0,2053],[0.0825,2120],[0.305,2120],[0,2439],[0.135,2484],[0.08499999999999999,2484],[0.042499999999999996,2689],[0,2803],[0.18,3021],[0,3262],[0.0375,3308],[0.145,3308],[0.08499999999999999,3496],[0.0725,3523],[0.16999999999999998,3523],[0,3697],[0.0375,3762],[0.1075,3762],[0.17500000000000002,3906],[0,4163],[0.1525,4181],[0.06999999999999999,4181],[0.16,4382],[0.12,4489],[0.065,4630],[0,4948],[0.02,5016],[0.012499999999999999,5016],[0,5163],[0,5290],[0,5522],[0.08750000000000001,5628],[0.125,5753],[0,5933],[0,6090],[0,6235],[0.1025,6475],[0.16249999999999998,6548],[0,6690],[0,6810],[0,6948],[0.02,6998],[0.10999999999999999,6998],[0.05499999999999999,7166],[0.05499999999999999,7218],[0,7218],[0.09499999999999999,7567],[0.09999999999999999,7589],[0.12,7589],[0,7757],[0.0925,7869],[0.21999999999999997,7980],[0,8122],[0.2225,8401],[0.027499999999999997,8470],[0.13999999999999999,8470],[0,8606],[0.1025,8766],[0.05499999999999999,8781],[0.0825,8781]])
surprise34 = np.array([[0,3831],[0.042499999999999996,3900],[0.145,3900],[0.11249999999999999,4139],[0.024999999999999998,4186],[0.27499999999999997,4186],[0.235,4472],[0.05249999999999999,4472],[0.145,4698],[0.05499999999999999,4823],[0.0625,4902],[0.06999999999999999,4902],[0,5148],[0,5392],[0,5665],[0.05499999999999999,5755],[0.005,5878],[0.075,6033],[0.065,6053],[0.0075,6053],[0.065,6259],[0,6370],[0.012499999999999999,6671],[0.024999999999999998,6671],[0.1225,6867],[0.13999999999999999,6892],[0.049999999999999996,6892],[0.027499999999999997,7124],[0,7274],[0.0625,7459],[0.015,7542],[0.08499999999999999,7672],[0.06,7767],[0.045,7986],[0.045,8007],[0.145,8007],[0.0825,8175],[0.08,8200],[0.08499999999999999,8200],[0.05249999999999999,8351],[0.05249999999999999,8376],[0.042499999999999996,8376],[0,8533],[0.04,8605],[0.11249999999999999,8605],[0.21749999999999997,8876],[0.024999999999999998,8945],[0.10999999999999999,8945],[0,9099],[0,9208],[0.05249999999999999,9271],[0.0225,9271],[0.042499999999999996,9466],[0.08499999999999999,9501],[0.13999999999999999,9501],[0.1225,9775],[0.02,9853],[0.13,9853],[0,10009],[0,10158],[0.0325,10249],[0.135,10249],[0.075,10382],[0,10670],[0.2425,10697],[0.047499999999999994,10697]])
surprise35 = np.array([[0,2687],[0.09499999999999999,2710],[0.012499999999999999,2710],[0.1025,3060],[0.22749999999999998,3065],[0.024999999999999998,3065],[0.2075,3270],[0.1525,3314],[0.11249999999999999,3314],[0.1675,3461],[0,3593],[0,3903],[0.24749999999999997,3970],[0.0675,3970],[0,4128],[0.05249999999999999,4175],[0,4175],[0.1275,4311],[0,4392],[0.135,4513],[0,4652],[0.195,4795],[0,4893],[0.16,4996],[0.255,5256],[0.135,5285],[0.0675,5285],[0.18249999999999997,5438],[0.155,5472],[0.0825,5472],[0.1775,5629],[0.125,5679],[0.0825,5679],[0.065,5819],[0.0575,5911],[0,6037],[0,6168],[0,6439],[0.17500000000000002,6475],[0,6475],[0.1475,6665],[0.075,6695],[0.0325,6695],[0.08,6850],[0.155,7036],[0.10499999999999998,7102],[0.08,7102],[0.0225,7212],[0.0725,7332],[0,7428],[0.034999999999999996,7517],[0.115,7517],[0,7647],[0.1425,7719],[0.17500000000000002,7719],[0.045,7853],[0.2425,7911],[0.115,7911],[0,8062],[0.0225,8137],[0.12,8137],[0,8279],[0.05249999999999999,8411],[0.0225,8486],[0.1475,8611],[0,8708]])
surprise36 = np.array([[0,1380],[0.185,1887],[0.13,2120],[0,2321],[0,2821],[0.1575,3272],[0.185,3488]])
surprise37 = np.array([[0.06999999999999999,1189],[0,1621],[0.08,2115],[0.0075,2339],[0.145,2554],[0.0725,2916],[0,3231],[0.1075,3755]])
surprise38 = np.array([[0.1575,1815],[0,2206],[0.1425,2767],[0.08,2958],[0.0625,3358],[0.09,3830],[0,4225]])
surprise39 = np.array([[0,1842],[0.1075,2284],[0,2750],[0,2969],[0,3175],[0.1425,3418],[0,3701]])
surprise40 = np.array([[0,1537],[0,2329],[0.10999999999999999,2570],[0.13,2786],[0.13249999999999998,3019],[0.03,3530],[0.024999999999999998,3973],[0.012499999999999999,4356]])
surprise41 = np.array([[0,1957],[0,1969],[0,1969],[0,2318],[0.235,2343],[0.065,2343],[0.23,2918],[0,2918]])
surprise42 = np.array([[0,2321],[0.03,2518],[0.115,2518],[0,2710],[0.0825,2718],[0.375,2718],[0.024999999999999998,2728],[0.32749999999999996,2728],[0.1075,2728],[0,3540],[0.012499999999999999,3729],[0.0775,3736],[0.012499999999999999,3736],[0.024999999999999998,3882],[0.02,3882]])
surprise43 = np.array([[0,6266],[0,6288],[0,6288],[0.02,6415],[0,6415],[0.08499999999999999,6415],[0,6517],[0,6517],[0,6523],[0.015,6523],[0,6523],[0,7675],[0.024999999999999998,7686],[0,7686],[0.08499999999999999,7875],[0,7875],[0.065,7875],[0,8133],[0.0225,8133],[0,8133]])
surprise44 = np.array([[0.0925,2442],[0.235,2460],[0,2460],[0.09999999999999999,2460],[0,3184],[0.13999999999999999,3206],[0.042499999999999996,3206],[0,3206],[0.02,3382],[0.02,3382],[0.02,3399],[0.0225,3399],[0,3399],[0,4281],[0.0375,4287],[0,4287],[0.017499999999999998,4718],[0,4718],[0.11249999999999999,4806],[0.0075,4806],[0.0025,4806]])
surprise45 = np.array([[0,2529],[0.0825,2554],[0.0575,2554],[0.115,2565],[0.02,2565],[0.1775,2565],[0.034999999999999996,2669],[0,2669],[0.08750000000000001,2682],[0,2682],[0.0825,2682],[0,3315],[0,3330],[0.04,3330],[0,3346],[0.06,3346],[0.0075,3346],[0.02,3485],[0.02,3508],[0,3508],[0.042499999999999996,3517],[0,3517],[0.042499999999999996,3517],[0,3825],[0,3849],[0,3849],[0.0075,3935],[0.02,3935],[0,3935]])
surprise46 = np.array([[0,4854],[0,5913],[0,6351],[0,7357],[0.05249999999999999,7856],[0,7990],[0.012499999999999999,8707],[0,10196],[0,10196],[0,10209],[0,10209],[0.1425,10209],[0.03,10768]])
surprise47 = np.array([[0.03,1510],[0.0075,2485],[0,3111],[0,3309],[0.0975,3793],[0,4586],[0.012499999999999999,5437],[0.0825,5963],[0,6829],[0.049999999999999996,7062],[0,7618],[0.049999999999999996,7626],[0,7626],[0.045,7649],[0.01,7649],[0.01,7649],[0,8954],[0.08750000000000001,8976],[0.0075,8976],[0,9604],[0.23249999999999998,10237],[0,10982],[0,11937],[0.1475,11952],[0,11952],[0,12515],[0.08,12524],[0,12524]])
surprise48 = np.array([[0,2647],[0,3263],[0.0775,3630],[0,4097],[0,4571],[0.135,4797],[0,5544],[0,5544],[0.425,5564],[0.0725,5564],[0.017499999999999998,5564],[0,6106],[0,6736],[0,7637],[0,8002],[0.2625,8543],[0.1175,9134],[0.31,9134],[0,9134],[0,9760],[0.24,10411],[0.02,10430],[0,10430],[0.02,10438],[0,10438],[0.0675,10438],[0,10960]])
surprise49 = np.array([[0,7533],[0.0925,8263],[0.11249999999999999,8469],[0.10999999999999999,8978],[0.027499999999999997,9570],[0.047499999999999994,9575],[0.18,9575],[0,10129],[0.0075,10689],[0.0825,10887],[0.18249999999999997,11338],[0,11338],[0.0225,11364],[0.017499999999999998,11364],[0,11364],[0,11845],[0,11994],[0.0925,12713],[0,13071],[0,13084],[0.05249999999999999,13084],[0,13499],[0.5275,13513],[0,13513],[0.0025,13513],[0.1925,13939],[0,14489]])
surprise50 = np.array([[0,1309],[0,1777],[0.065,2244],[0,2403],[0,2861],[0.1575,2861],[0.1475,2861],[0,3378],[0.045,3954],[0,4765],[0,5280],[0,6043],[0,6430],[0.185,7045],[0,7594],[0.0075,7600],[0.012499999999999999,7600],[0.18249999999999997,8197],[0.2975,8206],[0.17500000000000002,8206],[0.02,8212],[0.042499999999999996,8212],[0.13749999999999998,8212],[0.12,8905],[0.10499999999999998,9590]])


happy = [happy1,happy2,happy3,happy4,happy5,happy6,happy7,happy8,happy9,happy10,happy11,happy12,happy13,happy14,happy15,happy16,happy17,happy18,happy19,happy20,happy21,happy22,happy23,happy24,happy25,happy26,happy27,happy28,happy29,happy30,happy31,happy32,happy33,happy34,happy35,happy36,happy37,happy38,happy39,happy40,happy41,happy42,happy43,happy44,happy45,happy46,happy47,happy48,happy49,happy50]
sad = [sad1,sad2,sad3,sad4,sad6,sad7,sad8,sad9,sad10,sad11,sad12,sad13,sad14,sad15,sad16,sad17,sad18,sad19,sad20,sad21,sad22,sad23,sad24,sad25,sad26,sad27,sad28,sad29,sad30,sad31,sad32,sad33,sad34,sad35,sad36,sad37,sad38,sad39,sad40,sad41,sad42,sad43,sad44,sad45,sad46,sad47,sad48,sad49,sad50]
angry = [angry1,angry2,angry3,angry4,angry5,angry6,angry7,angry8,angry9,angry10,angry11,angry12,angry13,angry14,angry15,angry16,angry17,angry18,angry19,angry20,angry21,angry22,angry23,angry24,angry25,angry26,angry27,angry28,angry29,angry30,angry31,angry32,angry33,angry34,angry35,angry36,angry37,angry38,angry39,angry40,angry41,angry42,angry43,angry44,angry45,angry46,angry47,angry48,angry49,angry50]
surprise = [surprise1, surprise2, surprise3, surprise4, surprise5, surprise6, surprise7, surprise8, surprise9, surprise10, surprise11, surprise12, surprise13, surprise14, surprise15, surprise16, surprise17, surprise18, surprise19, surprise20, surprise21, surprise22, surprise23, surprise24, surprise25, surprise26, surprise27, surprise28, surprise29, surprise30, surprise31, surprise32, surprise33, surprise34, surprise35, surprise36, surprise37, surprise38, surprise39, surprise40, surprise41, surprise42, surprise43, surprise44, surprise45, surprise46, surprise47, surprise48, surprise49, surprise50]
disgust = [disgust2, disgust3, disgust4, disgust5, disgust11, disgust12, disgust13, disgust14, disgust15, disgust16, disgust17, disgust18, disgust19, disgust20, disgust21, disgust22, disgust23, disgust24, disgust25, disgust26, disgust27, disgust28, disgust29, disgust30, disgust31, disgust32, disgust33, disgust34, disgust35, disgust36, disgust37, disgust38, disgust39, disgust40, disgust41, disgust42, disgust43 ,disgust44, disgust45, disgust46, disgust47, disgust48, disgust49, disgust50]
fear = [fear1, fear2, fear3, fear4, fear5, fear7, fear8, fear9, fear10, fear11, fear12, fear13, fear14, fear15, fear16, fear17, fear18, fear19, fear20, fear21, fear22, fear23, fear24, fear25, fear26, fear27, fear28, fear29, fear30, fear31, fear32, fear33, fear34, fear35, fear36, fear37, fear38, fear39, fear40, fear41, fear42, fear43, fear44, fear45, fear46, fear47, fear48, fear49, fear50]

def fix(happylist):
  for i,j in enumerate(happylist):
    j= euclidean_distances(j,j)
    j= np.mean(j)
    happylist[i] = np.array([j])

from sklearn.metrics.pairwise import euclidean_distances

fix(happy)
fix(disgust)
fix(sad)
fix(angry)
fix(fear)
fix(surprise)

dataset = []
tags=[]
def fix2(list, tag):
  for i in list:
    dataset.append(i)
    tags.append(tag)

fix2(happy,0)
fix2(disgust,1)
fix2(sad,2)
fix2(angry,3)
fix2(fear,4)
fix2(surprise,5)

from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(dataset,tags)

def checkemotion(test):
  test = np.array([[0,21301],[0.17500000000000002,21534],[0.0225,21876],[0.0925,22067],[0.034999999999999996,22083],[0.1175,22336],[0.0775,22528],[0.042499999999999996,22855],[0.012499999999999999,22865],[0.08750000000000001,23019],[0.125,23131],[0.12,23391],[0.11249999999999999,23492],[0.1025,23586],[0.06999999999999999,23919],[0.02,24414],[0.06,24592],[0.06,24631]])
  test = euclidean_distances(test,test)
  test= np.mean(test)
  test = np.array([[test]])
  test = neigh.predict(test)
  labels = ["happy", "disgust", "sad", "angry", "fear", "surprise"]
  return labels[test[0]]


In [ ]:
import numpy as np
import gradio as gr


with gr.Blocks() as demo:
    gr.Markdown("hey kaba")
    with gr.Tab("create user"):
        email = gr.Textbox(label="email")
        password = gr.Textbox(label="password")
        user_output = gr.Textbox()
        user_button = gr.Button("create user")
    with gr.Tab("create post"):
        userid1 = gr.Number(label="userid", precision=0)
        textpost = gr.Textbox()
        post_output = gr.Textbox()
        post_button = gr.Button("create post")
    with gr.Tab("create comment"):
        userid2 = gr.Number(label="userid", precision=0)
        postid1 = gr.Number(label="postid", precision=0)
        text = gr.Textbox()
        comment_output = gr.Textbox()
        comment_button = gr.Button("create comment")
    with gr.Tab("follow"):
        userid3 = gr.Number(label="userid", precision=0)
        followid = gr.Number(label="followid", precision=0)
        follow_output = gr.Textbox()
        follow_button = gr.Button("follow")
    with gr.Tab("home"):
        userid4 = gr.Number(label="userid", precision=0)
        home_output = gr.Textbox()
        home_button = gr.Button("home")
    with gr.Tab("read post"):
        userid5 = gr.Number(label="userid", precision=0)
        postid2 = gr.Number(label="postid", precision=0)
        readpost_output = gr.Textbox()
        readpost_button = gr.Button("read post")
    with gr.Tab("read profile"):
        userid6 = gr.Number(label="userid", precision=0)
        profile_output = gr.Textbox()
        profile_button = gr.Button("read profile")
    with gr.Tab("following"):
        userid7 = gr.Number(label="userid", precision=0)
        following_output = gr.Textbox()
        following_button = gr.Button("following")
    with gr.Tab("followers"):
        userid8 = gr.Number(label="userid", precision=0)
        followers_output = gr.Textbox()
        followers_button = gr.Button("followers")
    with gr.Tab("get gif"):
        postid3 = gr.Number(label="postid", precision=0)
        getgif_output = gr.Image(show_download_button=True)
        getgif_button = gr.Button("get gif")
    with gr.Tab("get comment gif"):
        commentid = gr.Number(label="commentid", precision=0)
        getgif_output2 = gr.Image()
        getgif_button2 = gr.Button("get gif")
    with gr.Tab("Get touch emotion"):
        touch = gr.Textbox()
        touchoutput = gr.Textbox()
        touchbutton = gr.Button("get emotion")

    user_button.click(newuser, inputs=[email,password], outputs=user_output)
    post_button.click(create, inputs=[userid1,textpost], outputs=post_output)
    comment_button.click(createcomment, inputs=[userid2,text,postid1], outputs=comment_output)
    follow_button.click(follow, inputs=[userid3,followid], outputs=follow_output)
    home_button.click(home, inputs=[userid4], outputs=home_output)
    readpost_button.click(post, inputs=[userid5,postid2], outputs=readpost_output)
    profile_button.click(profile, inputs=[userid6], outputs=profile_output)
    following_button.click(following, inputs=[userid7], outputs=following_output)
    followers_button.click(followers, inputs=[userid8], outputs=followers_output)
    getgif_button.click(getgif, inputs=[postid3], outputs=getgif_output, preprocess=False, queue=True)
    getgif_button2.click(getcommentgif, inputs=[commentid], outputs=getgif_output2, preprocess=False,queue=True)
    touchbutton.click(checkemotion, inputs=[touch], outputs=touchoutput, queue=True)

demo.queue()
demo.launch(debug=True, inline=False)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://4c254aa0ae5d53f047.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
2023-11-10 05:06:31.220060: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-10 05:06:31.220106: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-10 05:06:31.220136: E tensorflow/compile

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/queueing.py", line 407, in call_prediction
    output = await route_utils.call_process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/route_utils.py", line 226, in call_process_api
    output = await app.get_blocks().process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1550, in process_api
    result = await self.call_function(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1185, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "/usr/local/lib/python3.10/dist-packages/anyio/to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 877, in run_sync_in_worker_thread
    return await future
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 807, in run
    re

2023-11-10 05:30:33.673960: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-10 05:30:33.674012: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-10 05:30:33.674041: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-10 05:30:34.906939: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Initializing the conversion map
Some weights of the model checkpoint at /content/drive/MyDrive/KabaModel2 were not used when initializing CLIPTextModel: ['text_model.encoder.layers.19.self_attn.out_proj.lora_B', 'text_model.encoder.layers

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/queueing.py", line 407, in call_prediction
    output = await route_utils.call_process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/route_utils.py", line 226, in call_process_api
    output = await app.get_blocks().process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1550, in process_api
    result = await self.call_function(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1185, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "/usr/local/lib/python3.10/dist-packages/anyio/to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 877, in run_sync_in_worker_thread
    return await future
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 807, in run
    re

2023-11-10 06:13:43.836614: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-10 06:13:43.836662: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-10 06:13:43.836689: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-10 06:13:44.951077: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Initializing the conversion map
Some weights of the model checkpoint at /content/drive/MyDrive/KabaModel2 were not used when initializing CLIPTextModel: ['text_model.encoder.layers.6.self_attn.v_proj.lora_A', 'text_model.encoder.layers.11

...

2023-11-10 06:45:10.445573: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-10 06:45:10.445629: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-10 06:45:10.445657: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-10 06:45:11.591315: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Initializing the conversion map
Some weights of the model checkpoint at /content/drive/MyDrive/KabaModel2 were not used when initializing CLIPTextModel: ['text_model.encoder.layers.7.mlp.fc2.lora_A', 'text_model.encoder.layers.2.self_attn

ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/protocols/websockets/websockets_impl.py", line 247, in run_asgi
    result = await self.app(self.scope, self.asgi_receive, self.asgi_send)
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/middleware/proxy_headers.py", line 84, in __call__
    return await self.app(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/fastapi/applications.py", line 1106, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/starlette/applications.py", line 122, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/starlette/middleware/errors.py", line 149, in __call__
    await self.app(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/starlette/middleware/cors.py", line 75, in __call__
    await self.app(scope, receive

2023-11-10 06:57:17.952940: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-10 06:57:17.952997: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-10 06:57:17.953027: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-10 06:57:19.071458: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Initializing the conversion map
Some weights of the model checkpoint at /content/drive/MyDrive/KabaModel2 were not used when initializing CLIPTextModel: ['text_model.encoder.layers.2.self_attn.k_proj.lora_A', 'text_model.encoder.layers.18

ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/protocols/websockets/websockets_impl.py", line 247, in run_asgi
    result = await self.app(self.scope, self.asgi_receive, self.asgi_send)
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/middleware/proxy_headers.py", line 84, in __call__
    return await self.app(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/fastapi/applications.py", line 1106, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/starlette/applications.py", line 122, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/starlette/middleware/errors.py", line 149, in __call__
    await self.app(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/starlette/middleware/cors.py", line 75, in __call__
    await self.app(scope, receive

2023-11-10 07:58:16.836170: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-10 07:58:16.836225: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-10 07:58:16.836271: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-10 07:58:17.955609: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Initializing the conversion map
Some weights of the model checkpoint at /content/drive/MyDrive/KabaModel2 were not used when initializing CLIPTextModel: ['text_model.encoder.layers.17.self_attn.v_proj.lora_A', 'text_model.encoder.layers.2

In [ ]:
print(posts)
print(comments)

[{'id': 0, 'user': 0, 'gif': '/content/videos/undefined, [object Object]0.gif', 'text': 'undefined, [object Object]', 'time': '09:07:17', 'comment': []}, {'id': 1, 'user': 0, 'gif': '/content/videos/kaba is good, [object Object]0.gif', 'text': 'kaba is good, [object Object]', 'time': '09:10:40', 'comment': []}, {'id': 2, 'user': 0, 'gif': '/content/videos/kkkk, [object Object]0.gif', 'text': 'kkkk, [object Object]', 'time': '09:27:47', 'comment': []}, {'id': 3, 'user': 0, 'gif': '/content/videos/kaba is busy, angry0.gif', 'text': 'kaba is busy, angry', 'time': '09:32:35', 'comment': []}, {'id': 4, 'user': 0, 'gif': '/content/videos/fffff, angry0.gif', 'text': 'fffff, angry', 'time': '09:36:04', 'comment': []}, {'id': 5, 'user': 0, 'gif': '/content/videos/mettya isogashii, angry0.gif', 'text': 'mettya isogashii, angry', 'time': '09:42:07', 'comment': []}, {'id': 6, 'user': 0, 'gif': '/content/videos/nemuina, angry0.gif', 'text': 'nemuina, angry', 'time': '09:49:30', 'comment': []}, {'id